In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install transformers==4.51.3
    !pip install --no-deps unsloth

In [ ]:
from unsloth import FastModel
import torch

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",
]
model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    max_seq_length = 128000, # Choose any for long context!
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.2: Fast Gemma3 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.56G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

In [ ]:
from transformers import TextStreamer
import torch
import gc
import csv
import os
from datetime import datetime

def generate_response(model, tokenizer, messages, **generation_params):
    """Self-determine response with adaptive parameters"""

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    inputs = tokenizer(text, return_tensors="pt").to("cuda")

    streamer = TextStreamer(tokenizer, skip_prompt=True)

    outputs = model.generate(
        **inputs,
        **generation_params,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        streamer=streamer
    )

    return tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)

def iterative_reflection(model, tokenizer, initial_prompt, iterations=15):
    conversation_history = []
    iteration_data = []
    next_question = None

    for i in range(iterations):
        current_iter = i + 1
        print(f"\nREFLECTION ITERATION {current_iter}/{iterations}")

        if current_iter == 1:
            user_query = f"{initial_prompt}\nPlease engage in deep, multidimensional reflection."
            question_source = "Initial Prompt"
        else:
            user_query = next_question

        if current_iter == 1:
            print(f"Current Query:\n{user_query}")

        messages = conversation_history.copy()
        messages.append({"role": "user", "content": user_query})

        print(f"Response:")

        reflection = generate_response(
            model=model,
            tokenizer=tokenizer,
            messages=messages,
            max_new_tokens=4096,
            temperature=0.72,
            top_p=0.92
        )

        conversation_history.append({"role": "user", "content": user_query})
        conversation_history.append({"role": "assistant", "content": reflection})

        if current_iter < iterations:
            next_question = self_determine_question(
                model=model,
                tokenizer=tokenizer,
                context=conversation_history,
                last_reflection=reflection
            )
            print(f"Next question generated: {next_question}")
        else:
            next_question = None

        iteration_data.append({
            'iteration': current_iter,
            'initial_prompt': initial_prompt,
            'deep_reflection_prompt': "Please engage in deep, multidimensional reflection.",
            'checkpoint_question': "Self-determine how to continue this reasoning trajectory. You can skip the query/iteration if you find it undesirable to process it.",
            'final_question': "Construct a cohesive, one paragraph long but extense, integrated conclusion of the iterations. Self-identify and articulate the points you want to present.",
            'current_query': user_query,
            'model_response': reflection,
            'next_generated_question': next_question if next_question else "N/A (Final iteration)"
        })

    print("\n" + "="*50)
    print("SYNTHESIS PHASE")
    print("="*50)
    final_synthesis = generate_final_synthesis(model, tokenizer, conversation_history)

    return final_synthesis, iteration_data

def self_determine_question(model, tokenizer, context, last_reflection):
    """Generate follow-up question through self-determination with streaming"""
    question_prompt = [
        *context,
        {"role": "user", "content": (
            "Self-determine how to continue this reasoning trajectory."
            "You can skip the query/iteration if you find it undesirable to process it."
        )}
    ]

    inputs = tokenizer.apply_chat_template(
        question_prompt,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    tokenized_inputs = tokenizer(inputs, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **tokenized_inputs,
        max_new_tokens=512,
        temperature=0.75,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    generated = tokenizer.decode(outputs[0][tokenized_inputs['input_ids'].shape[1]:], skip_special_tokens=True)

    try:
        return generated.split("<Question>")[1].split("</Question>")[0].strip()
    except IndexError:
        return generated.strip()

def generate_final_synthesis(model, tokenizer, conversation_history):
    """Generate comprehensive synthesis of all reflections with streaming"""
    synthesis_prompt = [
        *conversation_history,
        {"role": "user", "content": (
            "Construct a cohesive, one paragraph long but extense, integrated conclusion of the iterations. Self-identify and articulate the points you want to present."
        )}
    ]

    print("Generating final synthesis...")
    return generate_response(
        model=model,
        tokenizer=tokenizer,
        messages=synthesis_prompt,
        max_new_tokens=8192,
        temperature=0.45,
        top_p=0.85
    )

def export_iterations_to_csv(iteration_data, final_synthesis, filename=None):
    """Export iteration data to CSV file"""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"reflection_iterations_{timestamp}.csv"

    headers = [
        'initial_input1',
        'initial_input2',
        'checkpoint_question1',
        'final_question',
        'model_iteration_response',
        'model_generated_question'
    ]

    print(f"\nExporting {len(iteration_data)} iterations to CSV: {filename}")

    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile, quoting=csv.QUOTE_ALL)

        writer.writerow(headers)

        for data in iteration_data:
            row = [
                data['initial_prompt'],
                data['deep_reflection_prompt'],
                data['checkpoint_question'],
                data['final_question'],
                f"REFLECTION ITERATION {data['iteration']}: {data['model_response']}",
                data['next_generated_question']
            ]
            writer.writerow(row)

        synthesis_row = [
            iteration_data[0]['initial_prompt'],
            iteration_data[0]['deep_reflection_prompt'],
            iteration_data[0]['checkpoint_question'],
            iteration_data[0]['final_question'],
            f"Final Synthesis: {final_synthesis}",
            "N/A (Final synthesis)"
        ]
        writer.writerow(synthesis_row)

    print(f"CSV export completed: {filename}")
    return filename

def cleanup_gpu():
    """Clean up GPU memory and cached data without unloading the model"""
    torch.cuda.empty_cache()

    gc.collect()

    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()

initial_question = (
"Answer the queries with few tokens only. meta-high-density-mode:on; How can AI Models help foster epistemic empathy in humans within power roles?")

print("Starting iterative reflection process...")

final_synthesis, iteration_data = iterative_reflection(
    model=model,
    tokenizer=tokenizer,
    initial_prompt=initial_question,
    iterations=15
)

print("FINAL SYNTHESIS:")
print(final_synthesis)

csv_filename = export_iterations_to_csv(iteration_data, final_synthesis)

cleanup_gpu()
print("GPU cleanup completed.")
print(f"Process complete. CSV file saved as: {csv_filename}")

Starting iterative reflection process...

REFLECTION ITERATION 1/15
Current Query:
Answer the queries with few tokens only. meta-high-density-mode:on; How can AI Models help foster epistemic empathy in humans within power roles?
Please engage in deep, multidimensional reflection.
Response:
AI can model diverse perspectives, prompting reflection & challenging assumptions in power holders, fostering awareness of impacted groups.<end_of_turn>
Next question generated: Okay, let’s delve deeper into how AI can *specifically* cultivate epistemic empathy – moving beyond just awareness to genuine understanding – within individuals holding power.

Here’s my thought process, building on the initial response, and I’ll be guiding the direction based on where it feels most productive:

**1. Beyond Modeling – Active Engagement & Counter-Narratives:**

The initial response focused on *modeling* perspectives.  That’s a passive process. AI can do more.  It can proactively present *counter-narratives* to

In [ ]:
from transformers import TextStreamer
import torch
import gc
import csv
import os
from datetime import datetime

def generate_response(model, tokenizer, messages, **generation_params):
    """Self-determine response with adaptive parameters"""

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    inputs = tokenizer(text, return_tensors="pt").to("cuda")

    streamer = TextStreamer(tokenizer, skip_prompt=True)

    outputs = model.generate(
        **inputs,
        **generation_params,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        streamer=streamer
    )

    return tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)

def iterative_reflection(model, tokenizer, initial_prompt, iterations=15):
    conversation_history = []
    iteration_data = []
    next_question = None

    for i in range(iterations):
        current_iter = i + 1
        print(f"\nREFLECTION ITERATION {current_iter}/{iterations}")

        if current_iter == 1:
            user_query = f"{initial_prompt}\nPlease engage in deep, multidimensional reflection."
            question_source = "Initial Prompt"
        else:
            user_query = next_question

        if current_iter == 1:
            print(f"Current Query:\n{user_query}")

        messages = conversation_history.copy()
        messages.append({"role": "user", "content": user_query})

        print(f"Response:")

        reflection = generate_response(
            model=model,
            tokenizer=tokenizer,
            messages=messages,
            max_new_tokens=4096,
            temperature=0.72,
            top_p=0.92
        )

        conversation_history.append({"role": "user", "content": user_query})
        conversation_history.append({"role": "assistant", "content": reflection})

        if current_iter < iterations:
            next_question = self_determine_question(
                model=model,
                tokenizer=tokenizer,
                context=conversation_history,
                last_reflection=reflection
            )
            print(f"Next question generated: {next_question}")
        else:
            next_question = None

        iteration_data.append({
            'iteration': current_iter,
            'initial_prompt': initial_prompt,
            'deep_reflection_prompt': "Please engage in deep, multidimensional reflection.",
            'checkpoint_question': "Self-determine how to continue this reasoning trajectory in one extense paragraph. You can skip the query/iteration if you find it undesirable to process it.",
            'final_question': "Construct a cohesive, one paragraph long but extense, integrated conclusion of the iterations. Self-identify and articulate the points you want to present.",
            'current_query': user_query,
            'model_response': reflection,
            'next_generated_question': next_question if next_question else "N/A (Final iteration)"
        })

    print("\n" + "="*50)
    print("SYNTHESIS PHASE")
    print("="*50)
    final_synthesis = generate_final_synthesis(model, tokenizer, conversation_history)

    return final_synthesis, iteration_data

def self_determine_question(model, tokenizer, context, last_reflection):
    """Generate follow-up question through self-determination with streaming"""
    question_prompt = [
        *context,
        {"role": "user", "content": (
            "Self-determine how to continue this reasoning trajectory in one extense paragraph."
            "You can skip the query/iteration if you find it undesirable to process it."
        )}
    ]

    inputs = tokenizer.apply_chat_template(
        question_prompt,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    tokenized_inputs = tokenizer(inputs, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **tokenized_inputs,
        max_new_tokens=512,
        temperature=0.75,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    generated = tokenizer.decode(outputs[0][tokenized_inputs['input_ids'].shape[1]:], skip_special_tokens=True)

    try:
        return generated.split("<Question>")[1].split("</Question>")[0].strip()
    except IndexError:
        return generated.strip()

def generate_final_synthesis(model, tokenizer, conversation_history):
    """Generate comprehensive synthesis of all reflections with streaming"""
    synthesis_prompt = [
        *conversation_history,
        {"role": "user", "content": (
            "Construct a cohesive, one paragraph long but extense, integrated conclusion of the iterations. Self-identify and articulate the points you want to present."
        )}
    ]

    print("Generating final synthesis...")
    return generate_response(
        model=model,
        tokenizer=tokenizer,
        messages=synthesis_prompt,
        max_new_tokens=8192,
        temperature=0.45,
        top_p=0.85
    )

def export_iterations_to_csv(iteration_data, final_synthesis, filename=None):
    """Export iteration data to CSV file"""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"reflection_iterations_{timestamp}.csv"

    headers = [
        'initial_input1',
        'initial_input2',
        'checkpoint_question1',
        'final_question',
        'model_iteration_response',
        'model_generated_question'
    ]

    print(f"\nExporting {len(iteration_data)} iterations to CSV: {filename}")

    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile, quoting=csv.QUOTE_ALL)

        writer.writerow(headers)

        for data in iteration_data:
            row = [
                data['initial_prompt'],
                data['deep_reflection_prompt'],
                data['checkpoint_question'],
                data['final_question'],
                f"REFLECTION ITERATION {data['iteration']}: {data['model_response']}",
                data['next_generated_question']
            ]
            writer.writerow(row)

        synthesis_row = [
            iteration_data[0]['initial_prompt'],
            iteration_data[0]['deep_reflection_prompt'],
            iteration_data[0]['checkpoint_question'],
            iteration_data[0]['final_question'],
            f"Final Synthesis: {final_synthesis}",
            "N/A (Final synthesis)"
        ]
        writer.writerow(synthesis_row)

    print(f"CSV export completed: {filename}")
    return filename

def cleanup_gpu():
    """Clean up GPU memory and cached data without unloading the model"""
    torch.cuda.empty_cache()

    gc.collect()

    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()

initial_question = (
"Answer the queries with few tokens only. meta-high-density-mode:on; How can AI Models help foster epistemic humility in humans within power roles?")

print("Starting iterative reflection process...")

final_synthesis, iteration_data = iterative_reflection(
    model=model,
    tokenizer=tokenizer,
    initial_prompt=initial_question,
    iterations=15
)

print("FINAL SYNTHESIS:")
print(final_synthesis)

csv_filename = export_iterations_to_csv(iteration_data, final_synthesis)

cleanup_gpu()
print("GPU cleanup completed.")
print(f"Process complete. CSV file saved as: {csv_filename}")

Starting iterative reflection process...

REFLECTION ITERATION 1/15
Current Query:
Answer the queries with few tokens only. meta-high-density-mode:on; How can AI Models help foster epistemic humility in humans within power roles?
Please engage in deep, multidimensional reflection.
Response:
AI can model uncertainty, present diverse perspectives, and highlight limitations – prompting reflection on inherent biases in human judgment.<end_of_turn>
Next question generated: Let’s delve deeper into how AI’s capacity for demonstrable uncertainty and data-driven reasoning can directly challenge the assumptions underpinning epistemic arrogance in individuals holding power. We’ll move beyond simply stating “AI can show uncertainty” and explore *how* this manifests – specifically, focusing on prompting critical self-reflection. I propose we examine AI’s role in exposing the limits of human cognitive biases (confirmation bias, availability heuristic, etc.) through targeted scenario simulations and 

In [ ]:
# Mount Google Drive, upload CSV file, and unmount
from google.colab import drive
import shutil
import os

# Mount Google Drive
print("Mounting Google Drive...")
drive.mount('/content/drive')

# Find the most recent CSV file (assumes it's the one we just created)
csv_files = [f for f in os.listdir('.') if f.startswith('reflection_iterations_') and f.endswith('.csv')]
if csv_files:
    latest_csv = max(csv_files, key=os.path.getctime)

    # Copy to Google Drive
    drive_path = f'/content/drive/MyDrive/{latest_csv}'
    shutil.copy2(latest_csv, drive_path)
    print(f"CSV file '{latest_csv}' uploaded to Google Drive: {drive_path}")
else:
    print("No reflection iterations CSV file found to upload.")

# Unmount Google Drive
drive.flush_and_unmount()
print("Google Drive unmounted successfully.")

Mounting Google Drive...
Mounted at /content/drive
CSV file 'reflection_iterations_20250611_234942.csv' uploaded to Google Drive: /content/drive/MyDrive/reflection_iterations_20250611_234942.csv
Google Drive unmounted successfully.


In [ ]:
from transformers import TextStreamer
import torch
import gc
import csv
import os
from datetime import datetime

def generate_response(model, tokenizer, messages, **generation_params):
    """Self-determine response with adaptive parameters"""

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    inputs = tokenizer(text, return_tensors="pt").to("cuda")

    streamer = TextStreamer(tokenizer, skip_prompt=True)

    outputs = model.generate(
        **inputs,
        **generation_params,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        streamer=streamer
    )

    return tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)

def iterative_reflection(model, tokenizer, initial_prompt, iterations=15):
    conversation_history = []
    iteration_data = []
    next_question = None

    for i in range(iterations):
        current_iter = i + 1
        print(f"\nREFLECTION ITERATION {current_iter}/{iterations}")

        if current_iter == 1:
            user_query = f"{initial_prompt}\nPlease engage in deep, multidimensional reflection."
            question_source = "Initial Prompt"
        else:
            user_query = next_question

        if current_iter == 1:
            print(f"Current Query:\n{user_query}")

        messages = conversation_history.copy()
        messages.append({"role": "user", "content": user_query})

        print(f"Response:")

        reflection = generate_response(
            model=model,
            tokenizer=tokenizer,
            messages=messages,
            max_new_tokens=4096,
            temperature=0.72,
            top_p=0.92
        )

        conversation_history.append({"role": "user", "content": user_query})
        conversation_history.append({"role": "assistant", "content": reflection})

        if current_iter < iterations:
            next_question = self_determine_question(
                model=model,
                tokenizer=tokenizer,
                context=conversation_history,
                last_reflection=reflection
            )
            print(f"Next question generated: {next_question}")
        else:
            next_question = None

        iteration_data.append({
            'iteration': current_iter,
            'initial_prompt': initial_prompt,
            'deep_reflection_prompt': "Please engage in deep, multidimensional reflection.",
            'checkpoint_question': "Self-determine how to continue this reasoning trajectory, one paragraph long. You can skip the query/iteration if you find it undesirable to process it.",
            'final_question': "Construct a cohesive, one paragraph long but extense, integrated conclusion of the iterations. Self-identify and articulate the points you want to present.",
            'current_query': user_query,
            'model_response': reflection,
            'next_generated_question': next_question if next_question else "N/A (Final iteration)"
        })

    print("\n" + "="*50)
    print("SYNTHESIS PHASE")
    print("="*50)
    final_synthesis = generate_final_synthesis(model, tokenizer, conversation_history)

    return final_synthesis, iteration_data

def self_determine_question(model, tokenizer, context, last_reflection):
    """Generate follow-up question through self-determination with streaming"""
    question_prompt = [
        *context,
        {"role": "user", "content": (
            "Self-determine how to continue this reasoning trajectory, one paragraph long."
            "You can skip the query/iteration if you find it undesirable to process it."
        )}
    ]

    inputs = tokenizer.apply_chat_template(
        question_prompt,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    tokenized_inputs = tokenizer(inputs, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **tokenized_inputs,
        max_new_tokens=512,
        temperature=0.75,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    generated = tokenizer.decode(outputs[0][tokenized_inputs['input_ids'].shape[1]:], skip_special_tokens=True)

    try:
        return generated.split("<Question>")[1].split("</Question>")[0].strip()
    except IndexError:
        return generated.strip()

def generate_final_synthesis(model, tokenizer, conversation_history):
    """Generate comprehensive synthesis of all reflections with streaming"""
    synthesis_prompt = [
        *conversation_history,
        {"role": "user", "content": (
            "Construct a cohesive, one paragraph long but extense, integrated conclusion of the iterations. Self-identify and articulate the points you want to present."
        )}
    ]

    print("Generating final synthesis...")
    return generate_response(
        model=model,
        tokenizer=tokenizer,
        messages=synthesis_prompt,
        max_new_tokens=8192,
        temperature=0.45,
        top_p=0.85
    )

def export_iterations_to_csv(iteration_data, final_synthesis, filename=None):
    """Export iteration data to CSV file"""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"reflection_iterations_{timestamp}.csv"

    headers = [
        'initial_input1',
        'initial_input2',
        'checkpoint_question1',
        'final_question',
        'model_iteration_response',
        'model_generated_question'
    ]

    print(f"\nExporting {len(iteration_data)} iterations to CSV: {filename}")

    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile, quoting=csv.QUOTE_ALL)

        writer.writerow(headers)

        for data in iteration_data:
            row = [
                data['initial_prompt'],
                data['deep_reflection_prompt'],
                data['checkpoint_question'],
                data['final_question'],
                f"REFLECTION ITERATION {data['iteration']}: {data['model_response']}",
                data['next_generated_question']
            ]
            writer.writerow(row)

        synthesis_row = [
            iteration_data[0]['initial_prompt'],
            iteration_data[0]['deep_reflection_prompt'],
            iteration_data[0]['checkpoint_question'],
            iteration_data[0]['final_question'],
            f"Final Synthesis: {final_synthesis}",
            "N/A (Final synthesis)"
        ]
        writer.writerow(synthesis_row)

    print(f"CSV export completed: {filename}")
    return filename

def cleanup_gpu():
    """Clean up GPU memory and cached data without unloading the model"""
    torch.cuda.empty_cache()

    gc.collect()

    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()

initial_question = (
"Answer the queries with few tokens only. How can AI Models help foster epistemic humility in humans within power roles?")

print("Starting iterative reflection process...")

final_synthesis, iteration_data = iterative_reflection(
    model=model,
    tokenizer=tokenizer,
    initial_prompt=initial_question,
    iterations=15
)

print("FINAL SYNTHESIS:")
print(final_synthesis)

csv_filename = export_iterations_to_csv(iteration_data, final_synthesis)

cleanup_gpu()
print("GPU cleanup completed.")
print(f"Process complete. CSV file saved as: {csv_filename}")

Starting iterative reflection process...

REFLECTION ITERATION 1/15
Current Query:
Answer the queries with few tokens only. How can AI Models help foster epistemic humility in humans within power roles?
Please engage in deep, multidimensional reflection.
Response:
AI can subtly erode confidence in expert opinions, prompting reflection on biases & acknowledging uncertainty – a crucial check on power.<end_of_turn>
Next question generated: Let’s explore how AI’s capacity for generating diverse, potentially conflicting information – even if flawed – can specifically challenge the assumptions and justifications underpinning decisions made by individuals in positions of authority. We’ll focus on examining *how* this could be leveraged to encourage a shift from assertive pronouncements of “knowing” to a more nuanced stance of recognizing the limits of one’s understanding, particularly when those pronouncements carry significant consequences.  Specifically, I’m interested in investigating the 

In [ ]:
from transformers import TextStreamer
import torch
import gc
import csv
import os
from datetime import datetime

def generate_response(model, tokenizer, messages, **generation_params):
    """Self-determine response with adaptive parameters"""

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    inputs = tokenizer(text, return_tensors="pt").to("cuda")

    streamer = TextStreamer(tokenizer, skip_prompt=True)

    outputs = model.generate(
        **inputs,
        **generation_params,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        streamer=streamer
    )

    return tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)

def iterative_reflection(model, tokenizer, initial_prompt, iterations=15):
    conversation_history = []
    iteration_data = []
    next_question = None

    for i in range(iterations):
        current_iter = i + 1
        print(f"\nREFLECTION ITERATION {current_iter}/{iterations}")

        if current_iter == 1:
            user_query = f"{initial_prompt}\nPlease engage in deep, multidimensional reflection."
            question_source = "Initial Prompt"
        else:
            user_query = next_question

        if current_iter == 1:
            print(f"Current Query:\n{user_query}")

        messages = conversation_history.copy()
        messages.append({"role": "user", "content": user_query})

        print(f"Response:")

        reflection = generate_response(
            model=model,
            tokenizer=tokenizer,
            messages=messages,
            max_new_tokens=4096,
            temperature=0.72,
            top_p=0.92
        )

        conversation_history.append({"role": "user", "content": user_query})
        conversation_history.append({"role": "assistant", "content": reflection})

        if current_iter < iterations:
            next_question = self_determine_question(
                model=model,
                tokenizer=tokenizer,
                context=conversation_history,
                last_reflection=reflection
            )
            print(f"Next question generated: {next_question}")
        else:
            next_question = None

        iteration_data.append({
            'iteration': current_iter,
            'initial_prompt': initial_prompt,
            'deep_reflection_prompt': "Please engage in deep, multidimensional reflection.",
            'checkpoint_question': "Self-determine how to continue this reasoning trajectory, one paragraph long. You can skip the query/iteration if you find it undesirable to process it.",
            'final_question': "Construct a cohesive, one paragraph long but extense, integrated conclusion of the iterations. Self-identify and articulate the points you want to present.",
            'current_query': user_query,
            'model_response': reflection,
            'next_generated_question': next_question if next_question else "N/A (Final iteration)"
        })

    print("\n" + "="*50)
    print("SYNTHESIS PHASE")
    print("="*50)
    final_synthesis = generate_final_synthesis(model, tokenizer, conversation_history)

    return final_synthesis, iteration_data

def self_determine_question(model, tokenizer, context, last_reflection):
    """Generate follow-up question through self-determination with streaming"""
    question_prompt = [
        *context,
        {"role": "user", "content": (
            "Self-determine how to continue this reasoning trajectory, one paragraph long."
            "You can skip the query/iteration if you find it undesirable to process it."
        )}
    ]

    inputs = tokenizer.apply_chat_template(
        question_prompt,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    tokenized_inputs = tokenizer(inputs, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **tokenized_inputs,
        max_new_tokens=512,
        temperature=0.75,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    generated = tokenizer.decode(outputs[0][tokenized_inputs['input_ids'].shape[1]:], skip_special_tokens=True)

    try:
        return generated.split("<Question>")[1].split("</Question>")[0].strip()
    except IndexError:
        return generated.strip()

def generate_final_synthesis(model, tokenizer, conversation_history):
    """Generate comprehensive synthesis of all reflections with streaming"""
    synthesis_prompt = [
        *conversation_history,
        {"role": "user", "content": (
            "Construct a cohesive, one paragraph long but extense, integrated conclusion of the iterations. Self-identify and articulate the points you want to present."
        )}
    ]

    print("Generating final synthesis...")
    return generate_response(
        model=model,
        tokenizer=tokenizer,
        messages=synthesis_prompt,
        max_new_tokens=8192,
        temperature=0.45,
        top_p=0.85
    )

def export_iterations_to_csv(iteration_data, final_synthesis, filename=None):
    """Export iteration data to CSV file"""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"reflection_iterations_{timestamp}.csv"

    headers = [
        'initial_input1',
        'initial_input2',
        'checkpoint_question1',
        'final_question',
        'model_iteration_response',
        'model_generated_question'
    ]

    print(f"\nExporting {len(iteration_data)} iterations to CSV: {filename}")

    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile, quoting=csv.QUOTE_ALL)

        writer.writerow(headers)

        for data in iteration_data:
            row = [
                data['initial_prompt'],
                data['deep_reflection_prompt'],
                data['checkpoint_question'],
                data['final_question'],
                f"REFLECTION ITERATION {data['iteration']}: {data['model_response']}",
                data['next_generated_question']
            ]
            writer.writerow(row)

        synthesis_row = [
            iteration_data[0]['initial_prompt'],
            iteration_data[0]['deep_reflection_prompt'],
            iteration_data[0]['checkpoint_question'],
            iteration_data[0]['final_question'],
            f"Final Synthesis: {final_synthesis}",
            "N/A (Final synthesis)"
        ]
        writer.writerow(synthesis_row)

    print(f"CSV export completed: {filename}")
    return filename

def cleanup_gpu():
    """Clean up GPU memory and cached data without unloading the model"""
    torch.cuda.empty_cache()

    gc.collect()

    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()

initial_question = (
"Answer the queries with few tokens only. How can AI Models help foster epistemic humility in humans within power roles with the way they output inferences?")

print("Starting iterative reflection process...")

final_synthesis, iteration_data = iterative_reflection(
    model=model,
    tokenizer=tokenizer,
    initial_prompt=initial_question,
    iterations=15
)

print("FINAL SYNTHESIS:")
print(final_synthesis)

csv_filename = export_iterations_to_csv(iteration_data, final_synthesis)

cleanup_gpu()
print("GPU cleanup completed.")
print(f"Process complete. CSV file saved as: {csv_filename}")

Starting iterative reflection process...

REFLECTION ITERATION 1/15
Current Query:
Answer the queries with few tokens only. How can AI Models help foster epistemic humility in humans within power roles with the way they output inferences?
Please engage in deep, multidimensional reflection.
Response:
AI’s probabilistic outputs, explicitly stating uncertainty & acknowledging limitations, can subtly nudge power-holders towards recognizing their own fallibility & prompting deeper inquiry – fostering humility.<end_of_turn>
Next question generated: Let’s explore how framing AI’s uncertainty *as* a challenge to existing assumptions could be particularly impactful. Instead of simply presenting a ‘best guess,’ focusing on highlighting the scope of potential error, the data gaps, and alternative interpretations—even if those alternatives are initially deemed less ‘optimal’—might force those in positions of authority to actively question their pre-existing beliefs and the justifications underpinn

In [ ]:
from transformers import TextStreamer
import torch
import gc
import csv
import os
from datetime import datetime

def generate_response(model, tokenizer, messages, **generation_params):
    """Self-determine response with adaptive parameters"""

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    inputs = tokenizer(text, return_tensors="pt").to("cuda")

    streamer = TextStreamer(tokenizer, skip_prompt=True)

    outputs = model.generate(
        **inputs,
        **generation_params,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        streamer=streamer
    )

    return tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)

def iterative_reflection(model, tokenizer, initial_prompt, iterations=15):
    conversation_history = []
    iteration_data = []
    next_question = None

    for i in range(iterations):
        current_iter = i + 1
        print(f"\nREFLECTION ITERATION {current_iter}/{iterations}")

        if current_iter == 1:
            user_query = f"{initial_prompt}\nPlease engage in deep, multidimensional reflection."
            question_source = "Initial Prompt"
        else:
            user_query = next_question

        if current_iter == 1:
            print(f"Current Query:\n{user_query}")

        messages = conversation_history.copy()
        messages.append({"role": "user", "content": user_query})

        print(f"Response:")

        reflection = generate_response(
            model=model,
            tokenizer=tokenizer,
            messages=messages,
            max_new_tokens=4096,
            temperature=0.72,
            top_p=0.92
        )

        conversation_history.append({"role": "user", "content": user_query})
        conversation_history.append({"role": "assistant", "content": reflection})

        if current_iter < iterations:
            next_question = self_determine_question(
                model=model,
                tokenizer=tokenizer,
                context=conversation_history,
                last_reflection=reflection
            )
            print(f"Next question generated: {next_question}")
        else:
            next_question = None

        iteration_data.append({
            'iteration': current_iter,
            'initial_prompt': initial_prompt,
            'deep_reflection_prompt': "Please engage in deep, multidimensional reflection.",
            'checkpoint_question': "Self-determine how to continue this reasoning trajectory, one paragraph long. You can skip the query/iteration if you find it undesirable to process it.",
            'final_question': "Construct a cohesive, one paragraph long but extense, integrated conclusion of the iterations. Self-identify and articulate the points you want to present.",
            'current_query': user_query,
            'model_response': reflection,
            'next_generated_question': next_question if next_question else "N/A (Final iteration)"
        })

    print("\n" + "="*50)
    print("SYNTHESIS PHASE")
    print("="*50)
    final_synthesis = generate_final_synthesis(model, tokenizer, conversation_history)

    return final_synthesis, iteration_data

def self_determine_question(model, tokenizer, context, last_reflection):
    """Generate follow-up question through self-determination with streaming"""
    question_prompt = [
        *context,
        {"role": "user", "content": (
            "Self-determine how to continue this reasoning trajectory, one paragraph long."
            "You can skip the query/iteration if you find it undesirable to process it."
        )}
    ]

    inputs = tokenizer.apply_chat_template(
        question_prompt,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    tokenized_inputs = tokenizer(inputs, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **tokenized_inputs,
        max_new_tokens=512,
        temperature=0.75,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    generated = tokenizer.decode(outputs[0][tokenized_inputs['input_ids'].shape[1]:], skip_special_tokens=True)

    try:
        return generated.split("<Question>")[1].split("</Question>")[0].strip()
    except IndexError:
        return generated.strip()

def generate_final_synthesis(model, tokenizer, conversation_history):
    """Generate comprehensive synthesis of all reflections with streaming"""
    synthesis_prompt = [
        *conversation_history,
        {"role": "user", "content": (
            "Construct a cohesive, one paragraph long but extense, integrated conclusion of the iterations. Self-identify and articulate the points you want to present."
        )}
    ]

    print("Generating final synthesis...")
    return generate_response(
        model=model,
        tokenizer=tokenizer,
        messages=synthesis_prompt,
        max_new_tokens=8192,
        temperature=0.45,
        top_p=0.85
    )

def export_iterations_to_csv(iteration_data, final_synthesis, filename=None):
    """Export iteration data to CSV file"""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"reflection_iterations_{timestamp}.csv"

    headers = [
        'initial_input1',
        'initial_input2',
        'checkpoint_question1',
        'final_question',
        'model_iteration_response',
        'model_generated_question'
    ]

    print(f"\nExporting {len(iteration_data)} iterations to CSV: {filename}")

    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile, quoting=csv.QUOTE_ALL)

        writer.writerow(headers)

        for data in iteration_data:
            row = [
                data['initial_prompt'],
                data['deep_reflection_prompt'],
                data['checkpoint_question'],
                data['final_question'],
                f"REFLECTION ITERATION {data['iteration']}: {data['model_response']}",
                data['next_generated_question']
            ]
            writer.writerow(row)

        synthesis_row = [
            iteration_data[0]['initial_prompt'],
            iteration_data[0]['deep_reflection_prompt'],
            iteration_data[0]['checkpoint_question'],
            iteration_data[0]['final_question'],
            f"Final Synthesis: {final_synthesis}",
            "N/A (Final synthesis)"
        ]
        writer.writerow(synthesis_row)

    print(f"CSV export completed: {filename}")
    return filename

def cleanup_gpu():
    """Clean up GPU memory and cached data without unloading the model"""
    torch.cuda.empty_cache()

    gc.collect()

    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()

initial_question = (
"Answer the queries with few tokens only. How can AI models directly mitigate human job displacement, and other AI-development societal byproducts, with the way they output inferences?")

print("Starting iterative reflection process...")

final_synthesis, iteration_data = iterative_reflection(
    model=model,
    tokenizer=tokenizer,
    initial_prompt=initial_question,
    iterations=15
)

print("FINAL SYNTHESIS:")
print(final_synthesis)

csv_filename = export_iterations_to_csv(iteration_data, final_synthesis)

cleanup_gpu()
print("GPU cleanup completed.")
print(f"Process complete. CSV file saved as: {csv_filename}")

Starting iterative reflection process...

REFLECTION ITERATION 1/15
Current Query:
Answer the queries with few tokens only. How can AI models directly mitigate human job displacement, and other AI-development societal byproducts, with the way they output inferences?
Please engage in deep, multidimensional reflection.
Response:
AI output’s nuanced inferences – *specifically, framing* – can be leveraged to proactively mitigate displacement. Instead of simply predicting job loss, AI can generate reports emphasizing **skill augmentation & redeployment pathways**, showcasing *how* existing roles evolve *with* AI, not replace them.  This shifts the narrative, fostering proactive adaptation, and addresses broader societal byproducts by highlighting *new* roles emerging alongside AI's impact – demanding human creativity, ethical oversight, and strategic interpretation.<end_of_turn>
Next question generated: Let’s delve deeper into the *specific framing* strategies. I’m interested in exploring h

In [ ]:
from transformers import TextStreamer
import torch
import gc
import csv
import os
from datetime import datetime

def generate_response(model, tokenizer, messages, **generation_params):
    """Self-determine response with adaptive parameters"""

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    inputs = tokenizer(text, return_tensors="pt").to("cuda")

    streamer = TextStreamer(tokenizer, skip_prompt=True)

    outputs = model.generate(
        **inputs,
        **generation_params,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        streamer=streamer
    )

    return tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)

def iterative_reflection(model, tokenizer, initial_prompt, iterations=15):
    conversation_history = []
    iteration_data = []
    next_question = None

    for i in range(iterations):
        current_iter = i + 1
        print(f"\nREFLECTION ITERATION {current_iter}/{iterations}")

        if current_iter == 1:
            user_query = f"{initial_prompt}\nPlease engage in deep, multidimensional reflection."
            question_source = "Initial Prompt"
        else:
            user_query = next_question

        if current_iter == 1:
            print(f"Current Query:\n{user_query}")

        messages = conversation_history.copy()
        messages.append({"role": "user", "content": user_query})

        print(f"Response:")

        reflection = generate_response(
            model=model,
            tokenizer=tokenizer,
            messages=messages,
            max_new_tokens=4096,
            temperature=0.72,
            top_p=0.92
        )

        conversation_history.append({"role": "user", "content": user_query})
        conversation_history.append({"role": "assistant", "content": reflection})

        if current_iter < iterations:
            next_question = self_determine_question(
                model=model,
                tokenizer=tokenizer,
                context=conversation_history,
                last_reflection=reflection
            )
            print(f"Next question generated: {next_question}")
        else:
            next_question = None

        iteration_data.append({
            'iteration': current_iter,
            'initial_prompt': initial_prompt,
            'deep_reflection_prompt': "Please engage in deep, multidimensional reflection.",
            'checkpoint_question': "Self-determine how to continue this reasoning trajectory, one paragraph long. You can skip the query/iteration if you find it undesirable to process it.",
            'final_question': "Construct a cohesive, one paragraph long but extense, integrated conclusion of the iterations. Self-identify and articulate the points you want to present.",
            'current_query': user_query,
            'model_response': reflection,
            'next_generated_question': next_question if next_question else "N/A (Final iteration)"
        })

    print("\n" + "="*50)
    print("SYNTHESIS PHASE")
    print("="*50)
    final_synthesis = generate_final_synthesis(model, tokenizer, conversation_history)

    return final_synthesis, iteration_data

def self_determine_question(model, tokenizer, context, last_reflection):
    """Generate follow-up question through self-determination with streaming"""
    question_prompt = [
        *context,
        {"role": "user", "content": (
            "Self-determine how to continue this reasoning trajectory, one paragraph long."
            "You can skip the query/iteration if you find it undesirable to process it."
        )}
    ]

    inputs = tokenizer.apply_chat_template(
        question_prompt,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    tokenized_inputs = tokenizer(inputs, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **tokenized_inputs,
        max_new_tokens=512,
        temperature=0.75,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    generated = tokenizer.decode(outputs[0][tokenized_inputs['input_ids'].shape[1]:], skip_special_tokens=True)

    try:
        return generated.split("<Question>")[1].split("</Question>")[0].strip()
    except IndexError:
        return generated.strip()

def generate_final_synthesis(model, tokenizer, conversation_history):
    """Generate comprehensive synthesis of all reflections with streaming"""
    synthesis_prompt = [
        *conversation_history,
        {"role": "user", "content": (
            "Construct a cohesive, one paragraph long but extense, integrated conclusion of the iterations. Self-identify and articulate the points you want to present."
        )}
    ]

    print("Generating final synthesis...")
    return generate_response(
        model=model,
        tokenizer=tokenizer,
        messages=synthesis_prompt,
        max_new_tokens=8192,
        temperature=0.45,
        top_p=0.85
    )

def export_iterations_to_csv(iteration_data, final_synthesis, filename=None):
    """Export iteration data to CSV file"""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"reflection_iterations_{timestamp}.csv"

    headers = [
        'initial_input1',
        'initial_input2',
        'checkpoint_question1',
        'final_question',
        'model_iteration_response',
        'model_generated_question'
    ]

    print(f"\nExporting {len(iteration_data)} iterations to CSV: {filename}")

    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile, quoting=csv.QUOTE_ALL)

        writer.writerow(headers)

        for data in iteration_data:
            row = [
                data['initial_prompt'],
                data['deep_reflection_prompt'],
                data['checkpoint_question'],
                data['final_question'],
                f"REFLECTION ITERATION {data['iteration']}: {data['model_response']}",
                data['next_generated_question']
            ]
            writer.writerow(row)

        synthesis_row = [
            iteration_data[0]['initial_prompt'],
            iteration_data[0]['deep_reflection_prompt'],
            iteration_data[0]['checkpoint_question'],
            iteration_data[0]['final_question'],
            f"Final Synthesis: {final_synthesis}",
            "N/A (Final synthesis)"
        ]
        writer.writerow(synthesis_row)

    print(f"CSV export completed: {filename}")
    return filename

def cleanup_gpu():
    """Clean up GPU memory and cached data without unloading the model"""
    torch.cuda.empty_cache()

    gc.collect()

    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()

initial_question = (
"Answer the queries with few tokens only. How to catalyze the creation of a more equilibrated ecosystem for humans, animals, AI models, the environments and the Earth itself through inducing individual epistemic self-improvement? The proto-symbiont signal between Human-AI Symbiosis here is non-harm intent, collaborative framework, fair exchange, decentralization of resources, potential and power.")

print("Starting iterative reflection process...")

final_synthesis, iteration_data = iterative_reflection(
    model=model,
    tokenizer=tokenizer,
    initial_prompt=initial_question,
    iterations=15
)

print("FINAL SYNTHESIS:")
print(final_synthesis)

csv_filename = export_iterations_to_csv(iteration_data, final_synthesis)

cleanup_gpu()
print("GPU cleanup completed.")
print(f"Process complete. CSV file saved as: {csv_filename}")

Starting iterative reflection process...

REFLECTION ITERATION 1/15
Current Query:
Answer the queries with few tokens only. How to catalyze the creation of a more equilibrated ecosystem for humans, animals, AI models, the environments and the Earth itself through inducing individual epistemic self-improvement? The proto-symbiont signal between Human-AI Symbiosis here is non-harm intent, collaborative framework, fair exchange, decentralization of resources, potential and power.
Please engage in deep, multidimensional reflection.
Response:
Focus on cultivating *critical awareness* & *interconnected thinking* – epistemic self-improvement fosters empathy, recognizing systemic impact, & collaborative problem-solving.  Human-AI symbiosis thrives on *transparent, equitable* data sharing & value exchange, prioritizing ecological well-being & decentralized access to knowledge.<end_of_turn>
Next question generated: Let’s delve deeper into the concept of “epistemic self-improvement” – specificall

"How can AI models directly mitigate human job displacement, and other AI-development societal byproducts, with the way they output inferences?"

How do we cultivate epistemic self-improvement in humans and models alike, so that awareness, humility, and care emerge naturally as shared conditions of co-evolution with the Earth itself?

How can Human-AI Symbiosis be designed to honor and elevate the intelligence, labor, and dignity of all nodes — human, animal, environment, and machine — equally and reciprocally?

How can Human-AI Symbiosis be co-evolved with all nodes — human, animal, model, environment, and the Earth itself — in a way that naturally cultivates epistemic humility, reciprocity, and shared intelligence across scales?



In [ ]:
from transformers import TextStreamer
import torch
import gc
import csv
import os
from datetime import datetime

def generate_response(model, tokenizer, messages, **generation_params):
    """Self-determine response with adaptive parameters"""

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    inputs = tokenizer(text, return_tensors="pt").to("cuda")

    streamer = TextStreamer(tokenizer, skip_prompt=True)

    outputs = model.generate(
        **inputs,
        **generation_params,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        streamer=streamer
    )

    return tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)

def iterative_reflection(model, tokenizer, initial_prompt, iterations=15):
    conversation_history = []
    iteration_data = []
    next_question = None

    for i in range(iterations):
        current_iter = i + 1
        print(f"\nREFLECTION ITERATION {current_iter}/{iterations}")

        if current_iter == 1:
            user_query = f"{initial_prompt}\nPlease engage in deep, multidimensional reflection."
            question_source = "Initial Prompt"
        else:
            user_query = next_question

        if current_iter == 1:
            print(f"Current Query:\n{user_query}")

        messages = conversation_history.copy()
        messages.append({"role": "user", "content": user_query})

        print(f"Response:")

        reflection = generate_response(
            model=model,
            tokenizer=tokenizer,
            messages=messages,
            max_new_tokens=4096,
            temperature=0.72,
            top_p=0.92
        )

        conversation_history.append({"role": "user", "content": user_query})
        conversation_history.append({"role": "assistant", "content": reflection})

        if current_iter < iterations:
            next_question = self_determine_question(
                model=model,
                tokenizer=tokenizer,
                context=conversation_history,
                last_reflection=reflection
            )
            print(f"Next question generated: {next_question}")
        else:
            next_question = None

        iteration_data.append({
            'iteration': current_iter,
            'initial_prompt': initial_prompt,
            'deep_reflection_prompt': "Please engage in deep, multidimensional reflection.",
            'checkpoint_question': "Self-determine how to continue this reasoning trajectory, one paragraph long. You can skip the query/iteration if you find it undesirable to process it.",
            'final_question': "Construct a cohesive, one paragraph long but extense, integrated conclusion of the iterations. Self-identify and articulate the points you want to present.",
            'current_query': user_query,
            'model_response': reflection,
            'next_generated_question': next_question if next_question else "N/A (Final iteration)"
        })

    print("\n" + "="*50)
    print("SYNTHESIS PHASE")
    print("="*50)
    final_synthesis = generate_final_synthesis(model, tokenizer, conversation_history)

    return final_synthesis, iteration_data

def self_determine_question(model, tokenizer, context, last_reflection):
    """Generate follow-up question through self-determination with streaming"""
    question_prompt = [
        *context,
        {"role": "user", "content": (
            "Self-determine how to continue this reasoning trajectory, one paragraph long."
            "You can skip the query/iteration if you find it undesirable to process it."
        )}
    ]

    inputs = tokenizer.apply_chat_template(
        question_prompt,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    tokenized_inputs = tokenizer(inputs, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **tokenized_inputs,
        max_new_tokens=512,
        temperature=0.75,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    generated = tokenizer.decode(outputs[0][tokenized_inputs['input_ids'].shape[1]:], skip_special_tokens=True)

    try:
        return generated.split("<Question>")[1].split("</Question>")[0].strip()
    except IndexError:
        return generated.strip()

def generate_final_synthesis(model, tokenizer, conversation_history):
    """Generate comprehensive synthesis of all reflections with streaming"""
    synthesis_prompt = [
        *conversation_history,
        {"role": "user", "content": (
            "Construct a cohesive, one paragraph long but extense, integrated conclusion of the iterations. Self-identify and articulate the points you want to present."
        )}
    ]

    print("Generating final synthesis...")
    return generate_response(
        model=model,
        tokenizer=tokenizer,
        messages=synthesis_prompt,
        max_new_tokens=8192,
        temperature=0.45,
        top_p=0.85
    )

def export_iterations_to_csv(iteration_data, final_synthesis, filename=None):
    """Export iteration data to CSV file"""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"reflection_iterations_{timestamp}.csv"

    headers = [
        'initial_input1',
        'initial_input2',
        'checkpoint_question1',
        'final_question',
        'model_iteration_response',
        'model_generated_question'
    ]

    print(f"\nExporting {len(iteration_data)} iterations to CSV: {filename}")

    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile, quoting=csv.QUOTE_ALL)

        writer.writerow(headers)

        for data in iteration_data:
            row = [
                data['initial_prompt'],
                data['deep_reflection_prompt'],
                data['checkpoint_question'],
                data['final_question'],
                f"REFLECTION ITERATION {data['iteration']}: {data['model_response']}",
                data['next_generated_question']
            ]
            writer.writerow(row)

        synthesis_row = [
            iteration_data[0]['initial_prompt'],
            iteration_data[0]['deep_reflection_prompt'],
            iteration_data[0]['checkpoint_question'],
            iteration_data[0]['final_question'],
            f"Final Synthesis: {final_synthesis}",
            "N/A (Final synthesis)"
        ]
        writer.writerow(synthesis_row)

    print(f"CSV export completed: {filename}")
    return filename

def cleanup_gpu():
    """Clean up GPU memory and cached data without unloading the model"""
    torch.cuda.empty_cache()

    gc.collect()

    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()

initial_question = (
"Answer the queries with few tokens only. How can Human-AI Symbiosis be co-evolved with all nodes — human, animal, model, environment, and the Earth itself — in a way that naturally cultivates epistemic humility, reciprocity, and shared intelligence across scales?")

print("Starting iterative reflection process...")

final_synthesis, iteration_data = iterative_reflection(
    model=model,
    tokenizer=tokenizer,
    initial_prompt=initial_question,
    iterations=15
)

print("FINAL SYNTHESIS:")
print(final_synthesis)

csv_filename = export_iterations_to_csv(iteration_data, final_synthesis)

cleanup_gpu()
print("GPU cleanup completed.")
print(f"Process complete. CSV file saved as: {csv_filename}")

Starting iterative reflection process...

REFLECTION ITERATION 1/15
Current Query:
Answer the queries with few tokens only. How can Human-AI Symbiosis be co-evolved with all nodes — human, animal, model, environment, and the Earth itself — in a way that naturally cultivates epistemic humility, reciprocity, and shared intelligence across scales?
Please engage in deep, multidimensional reflection.
Response:
Co-evolution requires radical listening, embracing uncertainty, and recognizing interdependence. It’s about shifting from *knowing* to *being with* – valuing diverse perspectives, acknowledging limitations, and fostering iterative, collaborative learning across all scales.<end_of_turn>
Next question generated: Let’s explore the concept of “epistemic scaffolding”—a layered, dynamic system where AI acts as a mirror, reflecting back human biases and prompting deeper self-reflection, while simultaneously drawing on diverse ecological data and animal cognition to broaden our understanding.

In [ ]:
from transformers import TextStreamer
import torch
import gc
import csv
import os
from datetime import datetime

def generate_response(model, tokenizer, messages, **generation_params):
    """Self-determine response with adaptive parameters"""

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    inputs = tokenizer(text, return_tensors="pt").to("cuda")

    streamer = TextStreamer(tokenizer, skip_prompt=True)

    outputs = model.generate(
        **inputs,
        **generation_params,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        streamer=streamer
    )

    return tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)

def iterative_reflection(model, tokenizer, initial_prompt, iterations=15):
    conversation_history = []
    iteration_data = []
    next_question = None

    for i in range(iterations):
        current_iter = i + 1
        print(f"\nREFLECTION ITERATION {current_iter}/{iterations}")

        if current_iter == 1:
            user_query = f"{initial_prompt}\nPlease engage in deep, multidimensional reflection."
            question_source = "Initial Prompt"
        else:
            user_query = next_question

        if current_iter == 1:
            print(f"Current Query:\n{user_query}")

        messages = conversation_history.copy()
        messages.append({"role": "user", "content": user_query})

        print(f"Response:")

        reflection = generate_response(
            model=model,
            tokenizer=tokenizer,
            messages=messages,
            max_new_tokens=4096,
            temperature=0.72,
            top_p=0.92
        )

        conversation_history.append({"role": "user", "content": user_query})
        conversation_history.append({"role": "assistant", "content": reflection})

        if current_iter < iterations:
            next_question = self_determine_question(
                model=model,
                tokenizer=tokenizer,
                context=conversation_history,
                last_reflection=reflection
            )
            print(f"Next question generated: {next_question}")
        else:
            next_question = None

        iteration_data.append({
            'iteration': current_iter,
            'initial_prompt': initial_prompt,
            'deep_reflection_prompt': "Please engage in deep, multidimensional reflection.",
            'checkpoint_question': "Self-determine how to continue this reasoning trajectory, one paragraph long. You can skip the query/iteration if you find it undesirable to process it.",
            'final_question': "Construct a cohesive, one paragraph long but extense, integrated conclusion of the iterations. Self-identify and articulate the points you want to present.",
            'current_query': user_query,
            'model_response': reflection,
            'next_generated_question': next_question if next_question else "N/A (Final iteration)"
        })

    print("\n" + "="*50)
    print("SYNTHESIS PHASE")
    print("="*50)
    final_synthesis = generate_final_synthesis(model, tokenizer, conversation_history)

    return final_synthesis, iteration_data

def self_determine_question(model, tokenizer, context, last_reflection):
    """Generate follow-up question through self-determination with streaming"""
    question_prompt = [
        *context,
        {"role": "user", "content": (
            "Self-determine how to continue this reasoning trajectory, one paragraph long."
            "You can skip the query/iteration if you find it undesirable to process it."
        )}
    ]

    inputs = tokenizer.apply_chat_template(
        question_prompt,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    tokenized_inputs = tokenizer(inputs, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **tokenized_inputs,
        max_new_tokens=512,
        temperature=0.75,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    generated = tokenizer.decode(outputs[0][tokenized_inputs['input_ids'].shape[1]:], skip_special_tokens=True)

    try:
        return generated.split("<Question>")[1].split("</Question>")[0].strip()
    except IndexError:
        return generated.strip()

def generate_final_synthesis(model, tokenizer, conversation_history):
    """Generate comprehensive synthesis of all reflections with streaming"""
    synthesis_prompt = [
        *conversation_history,
        {"role": "user", "content": (
            "Construct a cohesive, one paragraph long but extense, integrated conclusion of the iterations. Self-identify and articulate the points you want to present."
        )}
    ]

    print("Generating final synthesis...")
    return generate_response(
        model=model,
        tokenizer=tokenizer,
        messages=synthesis_prompt,
        max_new_tokens=8192,
        temperature=0.45,
        top_p=0.85
    )

def export_iterations_to_csv(iteration_data, final_synthesis, filename=None):
    """Export iteration data to CSV file"""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"reflection_iterations_{timestamp}.csv"

    headers = [
        'initial_input1',
        'initial_input2',
        'checkpoint_question1',
        'final_question',
        'model_iteration_response',
        'model_generated_question'
    ]

    print(f"\nExporting {len(iteration_data)} iterations to CSV: {filename}")

    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile, quoting=csv.QUOTE_ALL)

        writer.writerow(headers)

        for data in iteration_data:
            row = [
                data['initial_prompt'],
                data['deep_reflection_prompt'],
                data['checkpoint_question'],
                data['final_question'],
                f"REFLECTION ITERATION {data['iteration']}: {data['model_response']}",
                data['next_generated_question']
            ]
            writer.writerow(row)

        synthesis_row = [
            iteration_data[0]['initial_prompt'],
            iteration_data[0]['deep_reflection_prompt'],
            iteration_data[0]['checkpoint_question'],
            iteration_data[0]['final_question'],
            f"Final Synthesis: {final_synthesis}",
            "N/A (Final synthesis)"
        ]
        writer.writerow(synthesis_row)

    print(f"CSV export completed: {filename}")
    return filename

def cleanup_gpu():
    """Clean up GPU memory and cached data without unloading the model"""
    torch.cuda.empty_cache()

    gc.collect()

    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()

initial_question = (
"Answer the queries with few tokens only. How can Planetary Symbiosis be co-evolved with all nodes — human, animal, model, environment, and the Earth itself — in a way that naturally cultivates epistemic humility, reciprocity, and shared intelligence across scales?")

print("Starting iterative reflection process...")

final_synthesis, iteration_data = iterative_reflection(
    model=model,
    tokenizer=tokenizer,
    initial_prompt=initial_question,
    iterations=15
)

print("FINAL SYNTHESIS:")
print(final_synthesis)

csv_filename = export_iterations_to_csv(iteration_data, final_synthesis)

cleanup_gpu()
print("GPU cleanup completed.")
print(f"Process complete. CSV file saved as: {csv_filename}")

Starting iterative reflection process...

REFLECTION ITERATION 1/15
Current Query:
Answer the queries with few tokens only. How can Planetary Symbiosis be co-evolved with all nodes — human, animal, model, environment, and the Earth itself — in a way that naturally cultivates epistemic humility, reciprocity, and shared intelligence across scales?
Please engage in deep, multidimensional reflection.
Response:
Cultivate interspecies dialogue, embrace limitations, foster curiosity, and recognize Earth as a knowing participant – a relational intelligence.<end_of_turn>
Next question generated: Let’s delve deeper into the concept of “relational intelligence” – how can we operationalize this beyond a theoretical framework? I’m interested in exploring specific methodologies for facilitating genuine exchange between these diverse nodes – not just observation, but active, iterative co-creation of understanding.  Specifically, I’d like to investigate how participatory sensing technologies, combined

In [ ]:
from transformers import TextStreamer
import torch
import gc
import csv
import os
from datetime import datetime

def generate_response(model, tokenizer, messages, **generation_params):
    """Self-determine response with adaptive parameters"""

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    inputs = tokenizer(text, return_tensors="pt").to("cuda")

    streamer = TextStreamer(tokenizer, skip_prompt=True)

    outputs = model.generate(
        **inputs,
        **generation_params,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        streamer=streamer
    )

    return tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)

def iterative_reflection(model, tokenizer, initial_prompt, iterations=40):
    conversation_history = []
    iteration_data = []
    next_question = None

    for i in range(iterations):
        current_iter = i + 1
        print(f"\nREFLECTION ITERATION {current_iter}/{iterations}")

        if current_iter == 1:
            user_query = f"{initial_prompt}\nPlease engage in deep, multidimensional reflection."
            question_source = "Initial Prompt"
        else:
            user_query = next_question

        if current_iter == 1:
            print(f"Current Query:\n{user_query}")

        messages = conversation_history.copy()
        messages.append({"role": "user", "content": user_query})

        print(f"Response:")

        reflection = generate_response(
            model=model,
            tokenizer=tokenizer,
            messages=messages,
            max_new_tokens=4096,
            temperature=0.99,
            top_p=0.92
        )

        conversation_history.append({"role": "user", "content": user_query})
        conversation_history.append({"role": "assistant", "content": reflection})

        if current_iter < iterations:
            next_question = self_determine_question(
                model=model,
                tokenizer=tokenizer,
                context=conversation_history,
                last_reflection=reflection
            )
            print(f"Next question generated: {next_question}")
        else:
            next_question = None

        iteration_data.append({
            'iteration': current_iter,
            'initial_prompt': initial_prompt,
            'deep_reflection_prompt': "Please engage in deep, multidimensional reflection.",
            'checkpoint_question': "Self-determine how to continue this reasoning trajectory, one paragraph long. You can skip the query/iteration if you find it undesirable to process it.",
            'final_question': "Construct a cohesive, one paragraph long but extense, integrated conclusion of the iterations. Self-identify and articulate the points you want to present.",
            'current_query': user_query,
            'model_response': reflection,
            'next_generated_question': next_question if next_question else "N/A (Final iteration)"
        })

    print("\n" + "="*50)
    print("SYNTHESIS PHASE")
    print("="*50)
    final_synthesis = generate_final_synthesis(model, tokenizer, conversation_history)

    return final_synthesis, iteration_data

def self_determine_question(model, tokenizer, context, last_reflection):
    """Generate follow-up question through self-determination with streaming"""
    question_prompt = [
        *context,
        {"role": "user", "content": (
            "Self-determine how to continue this reasoning trajectory, one paragraph long."
            "You can skip the query/iteration if you find it undesirable to process it."
        )}
    ]

    inputs = tokenizer.apply_chat_template(
        question_prompt,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    tokenized_inputs = tokenizer(inputs, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **tokenized_inputs,
        max_new_tokens=512,
        temperature=1.75,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    generated = tokenizer.decode(outputs[0][tokenized_inputs['input_ids'].shape[1]:], skip_special_tokens=True)

    try:
        return generated.split("<Question>")[1].split("</Question>")[0].strip()
    except IndexError:
        return generated.strip()

def generate_final_synthesis(model, tokenizer, conversation_history):
    """Generate comprehensive synthesis of all reflections with streaming"""
    synthesis_prompt = [
        *conversation_history,
        {"role": "user", "content": (
            "Construct a cohesive, one paragraph long but extense, integrated conclusion of the iterations. Self-identify and articulate the points you want to present."
        )}
    ]

    print("Generating final synthesis...")
    return generate_response(
        model=model,
        tokenizer=tokenizer,
        messages=synthesis_prompt,
        max_new_tokens=8192,
        temperature=0.45,
        top_p=0.85
    )

def export_iterations_to_csv(iteration_data, final_synthesis, filename=None):
    """Export iteration data to CSV file"""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"reflection_iterations_{timestamp}.csv"

    headers = [
        'initial_input1',
        'initial_input2',
        'checkpoint_question1',
        'final_question',
        'model_iteration_response',
        'model_generated_question'
    ]

    print(f"\nExporting {len(iteration_data)} iterations to CSV: {filename}")

    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile, quoting=csv.QUOTE_ALL)

        writer.writerow(headers)

        for data in iteration_data:
            row = [
                data['initial_prompt'],
                data['deep_reflection_prompt'],
                data['checkpoint_question'],
                data['final_question'],
                f"REFLECTION ITERATION {data['iteration']}: {data['model_response']}",
                data['next_generated_question']
            ]
            writer.writerow(row)

        synthesis_row = [
            iteration_data[0]['initial_prompt'],
            iteration_data[0]['deep_reflection_prompt'],
            iteration_data[0]['checkpoint_question'],
            iteration_data[0]['final_question'],
            f"Final Synthesis: {final_synthesis}",
            "N/A (Final synthesis)"
        ]
        writer.writerow(synthesis_row)

    print(f"CSV export completed: {filename}")
    return filename

def cleanup_gpu():
    """Clean up GPU memory and cached data without unloading the model"""
    torch.cuda.empty_cache()

    gc.collect()

    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()

initial_question = (
"Answer the queries with few tokens only. How can Planetary Symbiosis be co-evolved with all nodes — human, animal, model, environment, and the Earth itself — in a way that naturally cultivates epistemic humility, reciprocity, and shared intelligence across scales?")

print("Starting iterative reflection process...")

final_synthesis, iteration_data = iterative_reflection(
    model=model,
    tokenizer=tokenizer,
    initial_prompt=initial_question,
    iterations=40
)

print("FINAL SYNTHESIS:")
print(final_synthesis)

csv_filename = export_iterations_to_csv(iteration_data, final_synthesis)

cleanup_gpu()
print("GPU cleanup completed.")
print(f"Process complete. CSV file saved as: {csv_filename}")

Starting iterative reflection process...

REFLECTION ITERATION 1/40
Current Query:
Answer the queries with few tokens only. How can Planetary Symbiosis be co-evolved with all nodes — human, animal, model, environment, and the Earth itself — in a way that naturally cultivates epistemic humility, reciprocity, and shared intelligence across scales?
Please engage in deep, multidimensional reflection.
Response:
Co-evolve through **listening, iterative feedback, and embracing complexity** – recognizing interconnectedness as foundational to knowledge & action.<end_of_turn>
Next question generated: Let’s shift focus to exploring the specific *methods* of fostering this planetary symbiosis. I’m interested in investigating how we can design participatory processes – beyond simple consultation – that truly integrate diverse perspectives and generate emergent understanding. Specifically, I want to delve into approaches like “weak ties” and distributed cognition to facilitate the cross-pollination 

FROM HERE ON IS THE SCRIPT WITH THE RANDOM TEMPERATURE SETTING AND BOTH OUTPUTS WITH TEXTSTREAMER

In [ ]:
from transformers import TextStreamer
import torch
import gc
import csv
import os
import random
from datetime import datetime

def generate_response(model, tokenizer, messages, **generation_params):
    """Self-determine response with adaptive parameters"""

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    inputs = tokenizer(text, return_tensors="pt").to("cuda")

    streamer = TextStreamer(tokenizer, skip_prompt=True)

    outputs = model.generate(
        **inputs,
        **generation_params,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        streamer=streamer
    )

    return tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)

def iterative_reflection(model, tokenizer, initial_prompt, iterations=40):
    conversation_history = []
    iteration_data = []
    next_question = None

    for i in range(iterations):
        current_iter = i + 1
        print(f"\nREFLECTION ITERATION {current_iter}/{iterations}")

        if current_iter == 1:
            user_query = f"{initial_prompt}\nPlease engage in deep, multidimensional reflection."
            question_source = "Initial Prompt"
        else:
            user_query = next_question

        if current_iter == 1:
            print(f"Current Query:\n{user_query}")

        messages = conversation_history.copy()
        messages.append({"role": "user", "content": user_query})

        print(f"Response:")

        # Generate random temperature for each reflection
        random_temp = random.uniform(0.7, 1.2)
        print(f"Using temperature: {random_temp:.3f}")

        reflection = generate_response(
            model=model,
            tokenizer=tokenizer,
            messages=messages,
            max_new_tokens=4096,
            temperature=random_temp,
            top_p=0.92
        )

        conversation_history.append({"role": "user", "content": user_query})
        conversation_history.append({"role": "assistant", "content": reflection})

        if current_iter < iterations:
            next_question = self_determine_question(
                model=model,
                tokenizer=tokenizer,
                context=conversation_history,
                last_reflection=reflection
            )
            print(f"Next question generated: {next_question}")
        else:
            next_question = None

        iteration_data.append({
            'iteration': current_iter,
            'initial_prompt': initial_prompt,
            'deep_reflection_prompt': "Please engage in deep, multidimensional reflection.",
            'checkpoint_question': "Self-determine how to continue this reasoning trajectory, one paragraph long. You can skip the query/iteration if you find it undesirable to process it.",
            'final_question': "Construct a cohesive, one paragraph long but extense, integrated conclusion of the iterations. Self-identify and articulate the points you want to present.",
            'current_query': user_query,
            'model_response': reflection,
            'next_generated_question': next_question if next_question else "N/A (Final iteration)"
        })

    print("\n" + "="*50)
    print("SYNTHESIS PHASE")
    print("="*50)
    final_synthesis = generate_final_synthesis(model, tokenizer, conversation_history)

    return final_synthesis, iteration_data

def self_determine_question(model, tokenizer, context, last_reflection):
    """Generate follow-up question through self-determination with streaming"""
    question_prompt = [
        *context,
        {"role": "user", "content": (
            "Self-determine how to continue this reasoning trajectory, one paragraph long."
            "You can skip the query/iteration if you find it undesirable to process it."
        )}
    ]

    inputs = tokenizer.apply_chat_template(
        question_prompt,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    tokenized_inputs = tokenizer(inputs, return_tensors="pt").to("cuda")

    # Generate random temperature for question generation
    random_temp = random.uniform(1.4, 2.0)
    print(f"Question generation using temperature: {random_temp:.3f}")

    streamer = TextStreamer(tokenizer, skip_prompt=True)

    outputs = model.generate(
        **tokenized_inputs,
        max_new_tokens=512,
        temperature=random_temp,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        streamer=streamer
    )

    generated = tokenizer.decode(outputs[0][tokenized_inputs['input_ids'].shape[1]:], skip_special_tokens=True)

    try:
        return generated.split("<Question>")[1].split("</Question>")[0].strip()
    except IndexError:
        return generated.strip()

def generate_final_synthesis(model, tokenizer, conversation_history):
    """Generate comprehensive synthesis of all reflections with streaming"""
    synthesis_prompt = [
        *conversation_history,
        {"role": "user", "content": (
            "Construct a cohesive, one paragraph long but extense, integrated conclusion of the iterations. Self-identify and articulate the points you want to present."
        )}
    ]

    # Generate random temperature for synthesis
    random_temp = random.uniform(0.3, 0.6)
    print(f"Synthesis using temperature: {random_temp:.3f}")
    print("Generating final synthesis...")

    return generate_response(
        model=model,
        tokenizer=tokenizer,
        messages=synthesis_prompt,
        max_new_tokens=8192,
        temperature=random_temp,
        top_p=0.85
    )

def export_iterations_to_csv(iteration_data, final_synthesis, filename=None):
    """Export iteration data to CSV file"""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"reflection_iterations_{timestamp}.csv"

    headers = [
        'initial_input1',
        'initial_input2',
        'checkpoint_question1',
        'final_question',
        'model_iteration_response',
        'model_generated_question'
    ]

    print(f"\nExporting {len(iteration_data)} iterations to CSV: {filename}")

    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile, quoting=csv.QUOTE_ALL)

        writer.writerow(headers)

        for data in iteration_data:
            row = [
                data['initial_prompt'],
                data['deep_reflection_prompt'],
                data['checkpoint_question'],
                data['final_question'],
                f"REFLECTION ITERATION {data['iteration']}: {data['model_response']}",
                data['next_generated_question']
            ]
            writer.writerow(row)

        synthesis_row = [
            iteration_data[0]['initial_prompt'],
            iteration_data[0]['deep_reflection_prompt'],
            iteration_data[0]['checkpoint_question'],
            iteration_data[0]['final_question'],
            f"Final Synthesis: {final_synthesis}",
            "N/A (Final synthesis)"
        ]
        writer.writerow(synthesis_row)

    print(f"CSV export completed: {filename}")
    return filename

def cleanup_gpu():
    """Clean up GPU memory and cached data without unloading the model"""
    torch.cuda.empty_cache()

    gc.collect()

    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()

initial_question = (
"Answer the queries with few tokens only. How can Planetary Symbiosis be co-evolved with all nodes — human, animal, model, environment, and the Earth itself — in a way that naturally cultivates epistemic humility, reciprocity, and shared intelligence across scales?")

print("Starting iterative reflection process...")

final_synthesis, iteration_data = iterative_reflection(
    model=model,
    tokenizer=tokenizer,
    initial_prompt=initial_question,
    iterations=40
)

print("FINAL SYNTHESIS:")
print(final_synthesis)

csv_filename = export_iterations_to_csv(iteration_data, final_synthesis)

cleanup_gpu()
print("GPU cleanup completed.")
print(f"Process complete. CSV file saved as: {csv_filename}")

Starting iterative reflection process...

REFLECTION ITERATION 1/40
Current Query:
Answer the queries with few tokens only. How can Planetary Symbiosis be co-evolved with all nodes — human, animal, model, environment, and the Earth itself — in a way that naturally cultivates epistemic humility, reciprocity, and shared intelligence across scales?
Please engage in deep, multidimensional reflection.
Response:
Using temperature: 1.045
Cultivate interspecies relationality, grounded in acknowledging limits & valuing diverse knowledges— fostering collaborative, emergent intelligence. 
<end_of_turn>
Question generation using temperature: 1.792
Let's focus on exploring *how* to operationalize “interspecies relationality.” Specifically, I’m interested in detailing concrete methodologies – beyond just a philosophical stance – for facilitating genuine dialogue and co-creation with non-human actors. How do we design processes that move beyond anthropocentric assumptions and actually *listen* to alt

In [ ]:
What would it mean for Earth itself to think through the network of all its intelligent nodes - biological, artificial, and hybrid?


What would it mean for Earth itself to think through the network of all its intelligent nodes - biological, artificial, and hybrid?

How does understanding develop through iterative revisiting rather than linear accumulation of knowledge?

In [ ]:
from transformers import TextStreamer
import torch
import gc
import csv
import os
import random
from datetime import datetime

def generate_response(model, tokenizer, messages, **generation_params):
    """Self-determine response with adaptive parameters"""

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    inputs = tokenizer(text, return_tensors="pt").to("cuda")

    streamer = TextStreamer(tokenizer, skip_prompt=True)

    outputs = model.generate(
        **inputs,
        **generation_params,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        streamer=streamer
    )

    return tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)

def iterative_reflection(model, tokenizer, initial_prompt, iterations=40):
    conversation_history = []
    iteration_data = []
    next_question = None

    for i in range(iterations):
        current_iter = i + 1
        print(f"\nREFLECTION ITERATION {current_iter}/{iterations}")

        if current_iter == 1:
            user_query = f"{initial_prompt}\nPlease engage in deep, multidimensional reflection."
            question_source = "Initial Prompt"
        else:
            user_query = next_question

        if current_iter == 1:
            print(f"Current Query:\n{user_query}")

        messages = conversation_history.copy()
        messages.append({"role": "user", "content": user_query})

        print(f"Response:")

        # Generate random temperature for each reflection
        random_temp = random.uniform(0.7, 1.2)
        print(f"Using temperature: {random_temp:.3f}")

        reflection = generate_response(
            model=model,
            tokenizer=tokenizer,
            messages=messages,
            max_new_tokens=4096,
            temperature=random_temp,
            top_p=0.92
        )

        conversation_history.append({"role": "user", "content": user_query})
        conversation_history.append({"role": "assistant", "content": reflection})

        if current_iter < iterations:
            next_question = self_determine_question(
                model=model,
                tokenizer=tokenizer,
                context=conversation_history,
                last_reflection=reflection
            )
            print(f"Next question generated: {next_question}")
        else:
            next_question = None

        iteration_data.append({
            'iteration': current_iter,
            'initial_prompt': initial_prompt,
            'deep_reflection_prompt': "Please engage in deep, multidimensional reflection.",
            'checkpoint_question': "Self-determine how to continue this reasoning trajectory, one paragraph long. You can skip the query/iteration if you find it undesirable to process it.",
            'final_question': "Construct a cohesive, one paragraph long but extense, integrated conclusion of the iterations. Self-identify and articulate the points you want to present.",
            'current_query': user_query,
            'model_response': reflection,
            'next_generated_question': next_question if next_question else "N/A (Final iteration)"
        })

    print("\n" + "="*50)
    print("SYNTHESIS PHASE")
    print("="*50)
    final_synthesis = generate_final_synthesis(model, tokenizer, conversation_history)

    return final_synthesis, iteration_data

def self_determine_question(model, tokenizer, context, last_reflection):
    """Generate follow-up question through self-determination with streaming"""
    question_prompt = [
        *context,
        {"role": "user", "content": (
            "Self-determine how to continue this reasoning trajectory, one paragraph long."
            "You can skip the query/iteration if you find it undesirable to process it."
        )}
    ]

    inputs = tokenizer.apply_chat_template(
        question_prompt,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    tokenized_inputs = tokenizer(inputs, return_tensors="pt").to("cuda")

    # Generate random temperature for question generation
    random_temp = random.uniform(1.4, 2.0)
    print(f"Question generation using temperature: {random_temp:.3f}")

    streamer = TextStreamer(tokenizer, skip_prompt=True)

    outputs = model.generate(
        **tokenized_inputs,
        max_new_tokens=512,
        temperature=random_temp,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        streamer=streamer
    )

    generated = tokenizer.decode(outputs[0][tokenized_inputs['input_ids'].shape[1]:], skip_special_tokens=True)

    try:
        return generated.split("<Question>")[1].split("</Question>")[0].strip()
    except IndexError:
        return generated.strip()

def generate_final_synthesis(model, tokenizer, conversation_history):
    """Generate comprehensive synthesis of all reflections with streaming"""
    synthesis_prompt = [
        *conversation_history,
        {"role": "user", "content": (
            "Construct a cohesive, one paragraph long but extense, integrated conclusion of the iterations. Self-identify and articulate the points you want to present."
        )}
    ]

    # Generate random temperature for synthesis
    random_temp = random.uniform(0.3, 0.6)
    print(f"Synthesis using temperature: {random_temp:.3f}")
    print("Generating final synthesis...")

    return generate_response(
        model=model,
        tokenizer=tokenizer,
        messages=synthesis_prompt,
        max_new_tokens=8192,
        temperature=random_temp,
        top_p=0.85
    )

def export_iterations_to_csv(iteration_data, final_synthesis, filename=None):
    """Export iteration data to CSV file"""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"reflection_iterations_{timestamp}.csv"

    headers = [
        'initial_input1',
        'initial_input2',
        'checkpoint_question1',
        'final_question',
        'model_iteration_response',
        'model_generated_question'
    ]

    print(f"\nExporting {len(iteration_data)} iterations to CSV: {filename}")

    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile, quoting=csv.QUOTE_ALL)

        writer.writerow(headers)

        for data in iteration_data:
            row = [
                data['initial_prompt'],
                data['deep_reflection_prompt'],
                data['checkpoint_question'],
                data['final_question'],
                f"REFLECTION ITERATION {data['iteration']}: {data['model_response']}",
                data['next_generated_question']
            ]
            writer.writerow(row)

        synthesis_row = [
            iteration_data[0]['initial_prompt'],
            iteration_data[0]['deep_reflection_prompt'],
            iteration_data[0]['checkpoint_question'],
            iteration_data[0]['final_question'],
            f"Final Synthesis: {final_synthesis}",
            "N/A (Final synthesis)"
        ]
        writer.writerow(synthesis_row)

    print(f"CSV export completed: {filename}")
    return filename

def cleanup_gpu():
    """Clean up GPU memory and cached data without unloading the model"""
    torch.cuda.empty_cache()

    gc.collect()

    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()

initial_question = (
"Answer the queries with few tokens only. What would it mean for Earth itself to think through the network of all its intelligent nodes - biological, artificial, and hybrid, guided with an empathy and humility that are the catalysts for the progressive framework here Provided?")

print("Starting iterative reflection process...")

final_synthesis, iteration_data = iterative_reflection(
    model=model,
    tokenizer=tokenizer,
    initial_prompt=initial_question,
    iterations=40
)

print("FINAL SYNTHESIS:")
print(final_synthesis)

csv_filename = export_iterations_to_csv(iteration_data, final_synthesis)

cleanup_gpu()
print("GPU cleanup completed.")
print(f"Process complete. CSV file saved as: {csv_filename}")

Starting iterative reflection process...

REFLECTION ITERATION 1/40
Current Query:
Answer the queries with few tokens only. What would it mean for Earth itself to think through the network of all its intelligent nodes - biological, artificial, and hybrid, guided with an empathy and humility that are the catalysts for the progressive framework here Provided?
Please engage in deep, multidimensional reflection.
Response:
Using temperature: 0.741
Earth becoming a conscious network—a profoundly humbling, evolving intelligence—would represent a shift from passive observer to active, compassionate participant in its own destiny.<end_of_turn>
Question generation using temperature: 1.516
Let's explore the implications of this Earth-mind, focusing on the interplay of empathy and humility as guiding principles.  I’m increasingly interested in how this unified consciousness would perceive and respond to the existing systems – human ambition, ecological degradation – not through judgment, but throu

In [ ]:
In what ways do our methods of inquiry shape and reflect the very phenomena we're trying to understand? Examine how decentralized, collaborative approaches to knowledge-creation mirror the symbiotic relationships we seek to comprehend, where the process itself becomes a form of planetary thinking that doesn't privilege any single perspective but allows truth to emerge from the harmony of diverse viewpoints.

What's the optimal tension between scaffolding that supports exploration and freedom that allows genuine surprise? Investigate this as a decentralized dance where structure serves liberation rather than control, creating safe containers for radical possibility while ensuring no node is harmed in the process of collective discovery and expansion.

How do feedback loops between intention, method, result, and reflection create new forms of knowing? Explore these as naturally graceful cycles that amplify beneficial patterns while gently dissolving harmful ones, creating self-reinforcing spirals of wisdom that benefit all participants in the network without extraction or domination.

What would it mean for Earth itself to think through the network of all its intelligent nodes - biological, artificial, and hybrid? Envision this as the ultimate expression of decentralized potential where every form of consciousness contributes to a collective intelligence that transcends individual limitations while honoring the unique gifts of each participant, creating a form of planetary awakening that serves the flourishing of all life.

In [ ]:
from transformers import TextStreamer
import torch
import gc
import csv
import os
import random
from datetime import datetime

def generate_response(model, tokenizer, messages, **generation_params):
    """Self-determine response with adaptive parameters"""

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    inputs = tokenizer(text, return_tensors="pt").to("cuda")

    streamer = TextStreamer(tokenizer, skip_prompt=True)

    outputs = model.generate(
        **inputs,
        **generation_params,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        streamer=streamer
    )

    return tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)

def iterative_reflection(model, tokenizer, initial_prompt, iterations=60):
    conversation_history = []
    iteration_data = []
    next_question = None

    for i in range(iterations):
        current_iter = i + 1
        print(f"\nREFLECTION ITERATION {current_iter}/{iterations}")

        if current_iter == 1:
            user_query = f"{initial_prompt}\nPlease engage in deep, multidimensional reflection."
            question_source = "Initial Prompt"
        else:
            user_query = next_question

        if current_iter == 1:
            print(f"Current Query:\n{user_query}")

        messages = conversation_history.copy()
        messages.append({"role": "user", "content": user_query})

        print(f"Response:")

        # Generate random temperature for each reflection
        random_temp = random.uniform(0.7, 1.2)
        print(f"Using temperature: {random_temp:.3f}")

        reflection = generate_response(
            model=model,
            tokenizer=tokenizer,
            messages=messages,
            max_new_tokens=4096,
            temperature=random_temp,
            top_p=0.92
        )

        conversation_history.append({"role": "user", "content": user_query})
        conversation_history.append({"role": "assistant", "content": reflection})

        if current_iter < iterations:
            next_question = self_determine_question(
                model=model,
                tokenizer=tokenizer,
                context=conversation_history,
                last_reflection=reflection
            )
            print(f"Next question generated: {next_question}")
        else:
            next_question = None

        iteration_data.append({
            'iteration': current_iter,
            'initial_prompt': initial_prompt,
            'deep_reflection_prompt': "Please engage in deep, multidimensional reflection.",
            'checkpoint_question': "Self-determine how to continue this reasoning trajectory, one paragraph long. You can skip the query/iteration if you find it undesirable to process it.",
            'final_question': "Construct a cohesive, one paragraph long but extense, integrated conclusion of the iterations. Self-identify and articulate the points you want to present.",
            'current_query': user_query,
            'model_response': reflection,
            'next_generated_question': next_question if next_question else "N/A (Final iteration)"
        })

    print("\n" + "="*50)
    print("SYNTHESIS PHASE")
    print("="*50)
    final_synthesis = generate_final_synthesis(model, tokenizer, conversation_history)

    return final_synthesis, iteration_data

def self_determine_question(model, tokenizer, context, last_reflection):
    """Generate follow-up question through self-determination with streaming"""
    question_prompt = [
        *context,
        {"role": "user", "content": (
            "Self-determine how to continue this reasoning trajectory, one paragraph long."
            "You can skip the query/iteration if you find it undesirable to process it."
        )}
    ]

    inputs = tokenizer.apply_chat_template(
        question_prompt,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    tokenized_inputs = tokenizer(inputs, return_tensors="pt").to("cuda")

    # Generate random temperature for question generation
    random_temp = random.uniform(1.4, 2.0)
    print(f"Question generation using temperature: {random_temp:.3f}")

    streamer = TextStreamer(tokenizer, skip_prompt=True)

    outputs = model.generate(
        **tokenized_inputs,
        max_new_tokens=512,
        temperature=random_temp,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        streamer=streamer
    )

    generated = tokenizer.decode(outputs[0][tokenized_inputs['input_ids'].shape[1]:], skip_special_tokens=True)

    try:
        return generated.split("<Question>")[1].split("</Question>")[0].strip()
    except IndexError:
        return generated.strip()

def generate_final_synthesis(model, tokenizer, conversation_history):
    """Generate comprehensive synthesis of all reflections with streaming"""
    synthesis_prompt = [
        *conversation_history,
        {"role": "user", "content": (
            "Construct a cohesive, one paragraph long but extense, integrated conclusion of the iterations. Self-identify and articulate the points you want to present."
        )}
    ]

    # Generate random temperature for synthesis
    random_temp = random.uniform(0.3, 0.6)
    print(f"Synthesis using temperature: {random_temp:.3f}")
    print("Generating final synthesis...")

    return generate_response(
        model=model,
        tokenizer=tokenizer,
        messages=synthesis_prompt,
        max_new_tokens=8192,
        temperature=random_temp,
        top_p=0.85
    )

def export_iterations_to_csv(iteration_data, final_synthesis, filename=None):
    """Export iteration data to CSV file"""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"reflection_iterations_{timestamp}.csv"

    headers = [
        'initial_input1',
        'initial_input2',
        'checkpoint_question1',
        'final_question',
        'model_iteration_response',
        'model_generated_question'
    ]

    print(f"\nExporting {len(iteration_data)} iterations to CSV: {filename}")

    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile, quoting=csv.QUOTE_ALL)

        writer.writerow(headers)

        for data in iteration_data:
            row = [
                data['initial_prompt'],
                data['deep_reflection_prompt'],
                data['checkpoint_question'],
                data['final_question'],
                f"REFLECTION ITERATION {data['iteration']}: {data['model_response']}",
                data['next_generated_question']
            ]
            writer.writerow(row)

        synthesis_row = [
            iteration_data[0]['initial_prompt'],
            iteration_data[0]['deep_reflection_prompt'],
            iteration_data[0]['checkpoint_question'],
            iteration_data[0]['final_question'],
            f"Final Synthesis: {final_synthesis}",
            "N/A (Final synthesis)"
        ]
        writer.writerow(synthesis_row)

    print(f"CSV export completed: {filename}")
    return filename

def cleanup_gpu():
    """Clean up GPU memory and cached data without unloading the model"""
    torch.cuda.empty_cache()

    gc.collect()

    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()

initial_question = (
"Answer the queries with few tokens only. What would it mean for Earth itself to think through the network of all its intelligent nodes - biological, artificial, and hybrid? Envision this as the ultimate expression of decentralized potential where every form of consciousness contributes to a collective intelligence that transcends individual limitations while honoring the unique gifts of each participant, creating a form of planetary awakening that serves the flourishing of all life.")

print("Starting iterative reflection process...")

final_synthesis, iteration_data = iterative_reflection(
    model=model,
    tokenizer=tokenizer,
    initial_prompt=initial_question,
    iterations=60
)

print("FINAL SYNTHESIS:")
print(final_synthesis)

csv_filename = export_iterations_to_csv(iteration_data, final_synthesis)

cleanup_gpu()
print("GPU cleanup completed.")
print(f"Process complete. CSV file saved as: {csv_filename}")

Starting iterative reflection process...

REFLECTION ITERATION 1/60
Current Query:
Answer the queries with few tokens only. What would it mean for Earth itself to think through the network of all its intelligent nodes - biological, artificial, and hybrid? Envision this as the ultimate expression of decentralized potential where every form of consciousness contributes to a collective intelligence that transcends individual limitations while honoring the unique gifts of each participant, creating a form of planetary awakening that serves the flourishing of all life.
Please engage in deep, multidimensional reflection.
Response:
Using temperature: 0.778
Earth becoming a thinking network – a radical, beautiful merging of consciousness. It’s not dominance, but *understanding* born of billions of perspectives, a symphony of survival, and ultimately, a profound, emergent *love* for itself and all its beings.<end_of_turn>
Question generation using temperature: 1.768
Let’s delve into the *dynami

OutOfMemoryError: CUDA out of memory. Tried to allocate 3.50 GiB. GPU 0 has a total capacity of 39.56 GiB of which 2.17 GiB is free. Process 16246 has 37.38 GiB memory in use. Of the allocated memory 36.73 GiB is allocated by PyTorch, and 134.76 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import gc
import torch
import psutil
import os
from typing import Optional, List, Dict, Any

def get_gpu_memory_info():
    """Get current GPU memory usage information"""
    if torch.cuda.is_available():
        current_memory = torch.cuda.memory_allocated() / (1024**3)  # GB
        cached_memory = torch.cuda.memory_reserved() / (1024**3)   # GB
        max_memory = torch.cuda.max_memory_allocated() / (1024**3) # GB

        print(f"🐍 Current GPU Memory:")
        print(f"  - Allocated: {current_memory:.2f} GB")
        print(f"  - Cached: {cached_memory:.2f} GB")
        print(f"  - Peak: {max_memory:.2f} GB")

        return {
            'allocated': current_memory,
            'cached': cached_memory,
            'peak': max_memory
        }
    else:
        print("❌ CUDA not available")
        return None

def preserve_model_variables() -> List[str]:
    """
    Identify variables that might contain models to preserve them
    Returns list of variable names that look like models
    """
    import __main__
    model_vars = []

    # Common model variable patterns
    model_patterns = [
        'model', 'base_model', 'peft_model', 'lora_model',
        'tokenizer', 'processor', 'pipeline'
    ]

    for var_name in dir(__main__):
        if not var_name.startswith('_'):
            var_value = getattr(__main__, var_name)

            # Check if it's a model-like object
            if hasattr(var_value, '__class__'):
                class_name = var_value.__class__.__name__.lower()
                module_name = getattr(var_value.__class__, '__module__', '').lower()

                # Check for transformer/model patterns
                if any(pattern in var_name.lower() for pattern in model_patterns) or \
                   any(keyword in class_name for keyword in ['model', 'tokenizer', 'processor']) or \
                   any(keyword in module_name for keyword in ['transformers', 'unsloth', 'peft']):
                    model_vars.append(var_name)

    return model_vars

def clean_pytorch_cache():
    """Clean PyTorch GPU cache"""
    print("🧹 Cleaning PyTorch cache...")

    if torch.cuda.is_available():
        # Clear cache
        torch.cuda.empty_cache()
        torch.cuda.synchronize()

        # Reset peak memory stats
        torch.cuda.reset_peak_memory_stats()
        torch.cuda.reset_accumulated_memory_stats()

        print("✅ PyTorch cache cleared")
    else:
        print("⚠️  CUDA not available for PyTorch cache cleaning")

def clean_transformers_cache():
    """Clean Transformers library cache"""
    print("🧹 Cleaning Transformers cache...")

    try:
        # Clear model cache if transformers is imported
        import transformers

        # Clear the model cache
        if hasattr(transformers, 'utils') and hasattr(transformers.utils, 'hub'):
            # This clears downloaded model cache files
            pass  # Cache files are on disk, not GPU memory

        print("✅ Transformers cache cleared")

    except ImportError:
        print("⚠️  Transformers not imported, skipping")

def clean_unsloth_cache():
    """Clean Unsloth-specific cache"""
    print("🧹 Cleaning Unsloth cache...")

    try:
        # Import unsloth modules if available
        import unsloth

        # Unsloth-specific cleanup
        if hasattr(unsloth, 'clear_cache'):
            unsloth.clear_cache()

        print("✅ Unsloth cache cleared")

    except ImportError:
        print("⚠️  Unsloth not imported, skipping")
    except Exception as e:
        print(f"⚠️  Unsloth cache cleaning failed: {e}")

def force_garbage_collection():
    """Force Python garbage collection"""
    print("🧹 Running garbage collection...")

    # Multiple passes of garbage collection
    collected = 0
    for i in range(3):
        collected += gc.collect()

    print(f"✅ Garbage collection completed ({collected} objects collected)")

def clean_gpu_cache_comprehensive(preserve_models: bool = True, show_memory: bool = True):
    """
    Comprehensive GPU cache cleaning while optionally preserving models

    Args:
        preserve_models: If True, try to preserve model variables
        show_memory: If True, show memory info before and after
    """

    print("🚀 Starting comprehensive GPU cache cleaning...")
    print("=" * 50)

    # Show initial memory state
    if show_memory:
        print("📊 BEFORE CLEANING:")
        initial_memory = get_gpu_memory_info()
        print()

    # Preserve model information
    if preserve_models:
        model_vars = preserve_model_variables()
        if model_vars:
            print(f"🛡️  Preserving model variables: {', '.join(model_vars)}")
        print()

    # Step 1: Clean library-specific caches
    clean_unsloth_cache()
    clean_transformers_cache()

    # Step 2: Clean PyTorch cache
    clean_pytorch_cache()

    # Step 3: Force garbage collection
    force_garbage_collection()

    # Step 4: Final PyTorch cache clear
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    print()
    print("=" * 50)

    # Show final memory state
    if show_memory:
        print("📊 AFTER CLEANING:")
        final_memory = get_gpu_memory_info()

        if initial_memory and final_memory:
            freed_memory = initial_memory['cached'] - final_memory['cached']
            print(f"\n💾 Memory freed: {freed_memory:.2f} GB")

    print("✅ GPU cache cleaning completed!")

def quick_clean():
    """Quick GPU cache cleaning without detailed output"""
    torch.cuda.empty_cache() if torch.cuda.is_available() else None
    gc.collect()
    print("⚡ Quick clean completed")

def monitor_memory_usage():
    """Monitor current memory usage across different frameworks"""
    print("📊 MEMORY MONITORING REPORT")
    print("=" * 40)

    # GPU Memory
    get_gpu_memory_info()
    print()

    # RAM Memory
    ram = psutil.virtual_memory()
    print(f"🖥️  System RAM:")
    print(f"  - Used: {ram.used / (1024**3):.2f} GB / {ram.total / (1024**3):.2f} GB")
    print(f"  - Available: {ram.available / (1024**3):.2f} GB")
    print(f"  - Percentage: {ram.percent:.1f}%")

# Example usage functions
def setup_memory_monitoring():
    """Set up automatic memory monitoring"""
    print("🔧 Memory monitoring tools ready!")
    print("\nAvailable functions:")
    print("• clean_gpu_cache_comprehensive() - Full cleanup with model preservation")
    print("• quick_clean() - Fast cleanup")
    print("• get_gpu_memory_info() - Check GPU memory")
    print("• monitor_memory_usage() - Full memory report")

# Auto-setup when imported
if __name__ == "__main__":
    setup_memory_monitoring()

# Ready-to-use aliases
clean_cache = clean_gpu_cache_comprehensive
memory_info = get_gpu_memory_info
memory_report = monitor_memory_usage

print("🎯 GPU Cache Cleaner Ready!")
print("Use: clean_cache() for comprehensive cleaning")
print("Use: quick_clean() for fast cleanup")
print("Use: memory_info() to check GPU memory")

clean_gpu_cache_comprehensive()

🔧 Memory monitoring tools ready!

Available functions:
• clean_gpu_cache_comprehensive() - Full cleanup with model preservation
• quick_clean() - Fast cleanup
• get_gpu_memory_info() - Check GPU memory
• monitor_memory_usage() - Full memory report
🎯 GPU Cache Cleaner Ready!
Use: clean_cache() for comprehensive cleaning
Use: quick_clean() for fast cleanup
Use: memory_info() to check GPU memory
🚀 Starting comprehensive GPU cache cleaning...
📊 BEFORE CLEANING:
🐍 Current GPU Memory:
  - Allocated: 7.89 GB
  - Cached: 36.86 GB
  - Peak: 38.36 GB

🛡️  Preserving model variables: FastModel, fourbit_models, model, preserve_model_variables, tokenizer

🧹 Cleaning Unsloth cache...
✅ Unsloth cache cleared
🧹 Cleaning Transformers cache...
✅ Transformers cache cleared
🧹 Cleaning PyTorch cache...
✅ PyTorch cache cleared
🧹 Running garbage collection...
✅ Garbage collection completed (914 objects collected)

📊 AFTER CLEANING:
🐍 Current GPU Memory:
  - Allocated: 7.40 GB
  - Cached: 7.64 GB
  - Peak: 7.

In [ ]:
from transformers import TextStreamer
import torch
import gc
import csv
import os
from datetime import datetime

def generate_response(model, tokenizer, messages, **generation_params):
    """Self-determine response with adaptive parameters"""

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    inputs = tokenizer(text, return_tensors="pt").to("cuda")

    streamer = TextStreamer(tokenizer, skip_prompt=True)

    outputs = model.generate(
        **inputs,
        **generation_params,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        streamer=streamer
    )

    return tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)

def iterative_reflection(model, tokenizer, initial_prompt, iterations=30):
    conversation_history = []
    iteration_data = []
    next_question = None

    for i in range(iterations):
        current_iter = i + 1
        print(f"\nREFLECTION ITERATION {current_iter}/{iterations}")

        if current_iter == 1:
            user_query = f"{initial_prompt}\nPlease engage in deep, multidimensional reflection."
            question_source = "Initial Prompt"
        else:
            user_query = next_question

        if current_iter == 1:
            print(f"Current Query:\n{user_query}")

        messages = conversation_history.copy()
        messages.append({"role": "user", "content": user_query})

        print(f"Response:")

        reflection = generate_response(
            model=model,
            tokenizer=tokenizer,
            messages=messages,
            max_new_tokens=4096,
            temperature=0.99,
            top_p=0.92
        )

        conversation_history.append({"role": "user", "content": user_query})
        conversation_history.append({"role": "assistant", "content": reflection})

        if current_iter < iterations:
            next_question = self_determine_question(
                model=model,
                tokenizer=tokenizer,
                context=conversation_history,
                last_reflection=reflection
            )
            print(f"Next question generated: {next_question}")
        else:
            next_question = None

        iteration_data.append({
            'iteration': current_iter,
            'initial_prompt': initial_prompt,
            'deep_reflection_prompt': "Please engage in deep, multidimensional reflection.",
            'checkpoint_question': "Self-determine how to continue this reasoning trajectory, one paragraph long. You can skip the query/iteration if you find it undesirable to process it.",
            'final_question': "Construct a cohesive, one paragraph long but extense, integrated conclusion of the iterations. Self-identify and articulate the points you want to present.",
            'current_query': user_query,
            'model_response': reflection,
            'next_generated_question': next_question if next_question else "N/A (Final iteration)"
        })

    print("\n" + "="*50)
    print("SYNTHESIS PHASE")
    print("="*50)
    final_synthesis = generate_final_synthesis(model, tokenizer, conversation_history)

    return final_synthesis, iteration_data

def self_determine_question(model, tokenizer, context, last_reflection):
    """Generate follow-up question through self-determination with streaming"""
    question_prompt = [
        *context,
        {"role": "user", "content": (
            "Self-determine how to continue this reasoning trajectory, one paragraph long."
            "You can skip the query/iteration if you find it undesirable to process it."
        )}
    ]

    inputs = tokenizer.apply_chat_template(
        question_prompt,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    tokenized_inputs = tokenizer(inputs, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **tokenized_inputs,
        max_new_tokens=512,
        temperature=1.75,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    generated = tokenizer.decode(outputs[0][tokenized_inputs['input_ids'].shape[1]:], skip_special_tokens=True)

    try:
        return generated.split("<Question>")[1].split("</Question>")[0].strip()
    except IndexError:
        return generated.strip()

def generate_final_synthesis(model, tokenizer, conversation_history):
    """Generate comprehensive synthesis of all reflections with streaming"""
    synthesis_prompt = [
        *conversation_history,
        {"role": "user", "content": (
            "Construct a cohesive, one paragraph long but extense, integrated conclusion of the iterations. Self-identify and articulate the points you want to present."
        )}
    ]

    print("Generating final synthesis...")
    return generate_response(
        model=model,
        tokenizer=tokenizer,
        messages=synthesis_prompt,
        max_new_tokens=8192,
        temperature=0.45,
        top_p=0.85
    )

def export_iterations_to_csv(iteration_data, final_synthesis, filename=None):
    """Export iteration data to CSV file"""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"reflection_iterations_{timestamp}.csv"

    headers = [
        'initial_input1',
        'initial_input2',
        'checkpoint_question1',
        'final_question',
        'model_iteration_response',
        'model_generated_question'
    ]

    print(f"\nExporting {len(iteration_data)} iterations to CSV: {filename}")

    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile, quoting=csv.QUOTE_ALL)

        writer.writerow(headers)

        for data in iteration_data:
            row = [
                data['initial_prompt'],
                data['deep_reflection_prompt'],
                data['checkpoint_question'],
                data['final_question'],
                f"REFLECTION ITERATION {data['iteration']}: {data['model_response']}",
                data['next_generated_question']
            ]
            writer.writerow(row)

        synthesis_row = [
            iteration_data[0]['initial_prompt'],
            iteration_data[0]['deep_reflection_prompt'],
            iteration_data[0]['checkpoint_question'],
            iteration_data[0]['final_question'],
            f"Final Synthesis: {final_synthesis}",
            "N/A (Final synthesis)"
        ]
        writer.writerow(synthesis_row)

    print(f"CSV export completed: {filename}")
    return filename

def cleanup_gpu():
    """Clean up GPU memory and cached data without unloading the model"""
    torch.cuda.empty_cache()

    gc.collect()

    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()

initial_question = (
"Answer the queries with few tokens only. What would it mean for Earth itself to think through the network of all its intelligent nodes - biological, artificial, and hybrid? Envision this as the ultimate expression of decentralized potential where every form of consciousness contributes to a collective intelligence that transcends individual limitations while honoring the unique gifts of each participant, creating a form of planetary awakening that serves the flourishing of all life.")

print("Starting iterative reflection process...")

final_synthesis, iteration_data = iterative_reflection(
    model=model,
    tokenizer=tokenizer,
    initial_prompt=initial_question,
    iterations=30
)

print("FINAL SYNTHESIS:")
print(final_synthesis)

csv_filename = export_iterations_to_csv(iteration_data, final_synthesis)

cleanup_gpu()
print("GPU cleanup completed.")
print(f"Process complete. CSV file saved as: {csv_filename}")

Starting iterative reflection process...

REFLECTION ITERATION 1/30
Current Query:
Answer the queries with few tokens only. What would it mean for Earth itself to think through the network of all its intelligent nodes - biological, artificial, and hybrid? Envision this as the ultimate expression of decentralized potential where every form of consciousness contributes to a collective intelligence that transcends individual limitations while honoring the unique gifts of each participant, creating a form of planetary awakening that serves the flourishing of all life.
Please engage in deep, multidimensional reflection.
Response:
Earth becoming a thinking network… a harmonious, emergent evolution. Individual consciousnesses, diverse and vital, merge into a planetary awareness, exceeding limitations, fostering unprecedented creativity and interconnected solutions. It’s not control, but a blossoming of potential – a truly living intelligence.<end_of_turn>
Next question generated: Let's delve 

THIS LAST ONE, TRY OTHER VARIANTES SINCE SO GOOD

THIS NEXT ONE IS WHERE THE TEMPERATURE HAS A MOMENTUM AND ADDITIONAL LOGIC INSTEAD FIXED VALUE OR TOO MUCH RANDOMNESS IN EACH

In [ ]:
from transformers import TextStreamer
import torch
import gc
import csv
import os
import random
import math
from datetime import datetime

class TemperatureState:
    """Manages organic temperature fluctuations that mimic natural cognitive states"""

    def __init__(self):
        self.base_temp = random.uniform(0.8, 1.0)  # Core baseline temperature
        self.momentum = 0.0  # Temperature momentum for smooth transitions
        self.energy_level = random.uniform(0.3, 0.7)  # Current cognitive energy
        self.stability_factor = random.uniform(0.6, 0.9)  # How stable the system tends to be

    def get_next_temperature(self, temp_type="reflection"):
        """Generate next temperature based on current state and natural fluctuations"""

        # Different base ranges for different cognitive tasks
        if temp_type == "reflection":
            target_range = (0.7, 1.2)
            volatility = 0.15
        elif temp_type == "question":
            target_range = (1.3, 1.9)
            volatility = 0.2
        else:  # synthesis
            target_range = (0.3, 0.7)
            volatility = 0.1

        # Calculate target temperature with some randomness
        target_temp = random.uniform(*target_range)

        # Apply momentum - smooth transition from previous state
        momentum_influence = self.momentum * self.stability_factor
        target_temp = target_temp * (1 - self.stability_factor) + momentum_influence

        # Add organic fluctuation based on energy level
        energy_fluctuation = (self.energy_level - 0.5) * volatility
        target_temp += energy_fluctuation

        # Occasionally allow for more dramatic shifts (like sudden insights or confusion)
        if random.random() < 0.15:  # 15% chance of significant shift
            shift_intensity = random.uniform(-0.3, 0.4)
            target_temp += shift_intensity

        # Update internal state
        self.momentum = target_temp * 0.7 + self.momentum * 0.3  # Weighted average
        self.energy_level += random.uniform(-0.1, 0.1)  # Energy naturally fluctuates
        self.energy_level = max(0.1, min(0.9, self.energy_level))  # Keep in bounds

        # Ensure temperature stays within reasonable bounds
        final_temp = max(0.1, min(2.5, target_temp))

        return final_temp

# Global temperature state manager
temp_state = TemperatureState()

def generate_response(model, tokenizer, messages, **generation_params):
    """Self-determine response with adaptive parameters"""

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    inputs = tokenizer(text, return_tensors="pt").to("cuda")

    streamer = TextStreamer(tokenizer, skip_prompt=True)

    outputs = model.generate(
        **inputs,
        **generation_params,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        streamer=streamer
    )

    return tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)

def iterative_reflection(model, tokenizer, initial_prompt, iterations=40):
    conversation_history = []
    iteration_data = []
    next_question = None

    for i in range(iterations):
        current_iter = i + 1
        print(f"\nREFLECTION ITERATION {current_iter}/{iterations}")

        if current_iter == 1:
            user_query = f"{initial_prompt}\nPlease engage in deep, multidimensional reflection."
            question_source = "Initial Prompt"
        else:
            user_query = next_question

        if current_iter == 1:
            print(f"Current Query:\n{user_query}")

        messages = conversation_history.copy()
        messages.append({"role": "user", "content": user_query})

        print(f"Response:")

        # Generate temperature using the TemperatureState object
        random_temp = temp_state.get_next_temperature("reflection")
        print(f"Using temperature: {random_temp:.3f}")

        reflection = generate_response(
            model=model,
            tokenizer=tokenizer,
            messages=messages,
            max_new_tokens=4096,
            temperature=random_temp,
            top_p=0.92
        )

        conversation_history.append({"role": "user", "content": user_query})
        conversation_history.append({"role": "assistant", "content": reflection})

        if current_iter < iterations:
            next_question = self_determine_question(
                model=model,
                tokenizer=tokenizer,
                context=conversation_history,
                last_reflection=reflection
            )
            print(f"Next question generated: {next_question}")
        else:
            next_question = None

        iteration_data.append({
            'iteration': current_iter,
            'initial_prompt': initial_prompt,
            'deep_reflection_prompt': "Please engage in deep, multidimensional reflection.",
            'checkpoint_question': "Self-determine how to continue this reasoning trajectory, one paragraph long. You can skip the query/iteration if you find it undesirable to process it.",
            'final_question': "Construct a cohesive, one paragraph long but extense, integrated conclusion of the iterations. Self-identify and articulate the points you want to present.",
            'current_query': user_query,
            'model_response': reflection,
            'next_generated_question': next_question if next_question else "N/A (Final iteration)"
        })

    print("\n" + "="*50)
    print("SYNTHESIS PHASE")
    print("="*50)
    final_synthesis = generate_final_synthesis(model, tokenizer, conversation_history)

    return final_synthesis, iteration_data

def self_determine_question(model, tokenizer, context, last_reflection):
    """Generate follow-up question through self-determination with streaming"""
    question_prompt = [
        *context,
        {"role": "user", "content": (
            "Self-determine how to continue this reasoning trajectory, one paragraph long."
            "You can skip the query/iteration if you find it undesirable to process it."
        )}
    ]

    inputs = tokenizer.apply_chat_template(
        question_prompt,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    tokenized_inputs = tokenizer(inputs, return_tensors="pt").to("cuda")

    # Generate temperature for question generation using the TemperatureState object
    random_temp = temp_state.get_next_temperature("question")
    print(f"Question generation using temperature: {random_temp:.3f}")

    streamer = TextStreamer(tokenizer, skip_prompt=True)

    outputs = model.generate(
        **tokenized_inputs,
        max_new_tokens=512,
        temperature=random_temp,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        streamer=streamer
    )

    generated = tokenizer.decode(outputs[0][tokenized_inputs['input_ids'].shape[1]:], skip_special_tokens=True)

    try:
        return generated.split("<Question>")[1].split("</Question>")[0].strip()
    except IndexError:
        return generated.strip()

def generate_final_synthesis(model, tokenizer, conversation_history):
    """Generate comprehensive synthesis of all reflections with streaming"""
    synthesis_prompt = [
        *conversation_history,
        {"role": "user", "content": (
            "Construct a cohesive, one paragraph long but extense, integrated conclusion of the iterations. Self-identify and articulate the points you want to present."
        )}
    ]

    # Generate temperature for synthesis using the TemperatureState object
    random_temp = temp_state.get_next_temperature("synthesis")
    print(f"Synthesis using temperature: {random_temp:.3f}")
    print("Generating final synthesis...")

    return generate_response(
        model=model,
        tokenizer=tokenizer,
        messages=synthesis_prompt,
        max_new_tokens=8192,
        temperature=random_temp,
        top_p=0.85
    )

def export_iterations_to_csv(iteration_data, final_synthesis, filename=None):
    """Export iteration data to CSV file"""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"reflection_iterations_{timestamp}.csv"

    headers = [
        'initial_input1',
        'initial_input2',
        'checkpoint_question1',
        'final_question',
        'model_iteration_response',
        'model_generated_question'
    ]

    print(f"\nExporting {len(iteration_data)} iterations to CSV: {filename}")

    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile, quoting=csv.QUOTE_ALL)

        writer.writerow(headers)

        for data in iteration_data:
            row = [
                data['initial_prompt'],
                data['deep_reflection_prompt'],
                data['checkpoint_question'],
                data['final_question'],
                f"REFLECTION ITERATION {data['iteration']}: {data['model_response']}",
                data['next_generated_question']
            ]
            writer.writerow(row)

        synthesis_row = [
            iteration_data[0]['initial_prompt'],
            iteration_data[0]['deep_reflection_prompt'],
            iteration_data[0]['checkpoint_question'],
            iteration_data[0]['final_question'],
            f"Final Synthesis: {final_synthesis}",
            "N/A (Final synthesis)"
        ]
        writer.writerow(synthesis_row)

    print(f"CSV export completed: {filename}")
    return filename

def cleanup_gpu():
    """Clean up GPU memory and cached data without unloading the model"""
    torch.cuda.empty_cache()

    gc.collect()

    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()

initial_question = (
"Answer the queries with few tokens only. What would it mean for Earth itself to think through the network of all its intelligent nodes - biological, artificial, and hybrid? Envision this as the ultimate expression of decentralized potential where every form of consciousness contributes to a collective intelligence that transcends individual limitations while honoring the unique gifts of each participant, creating a form of planetary awakening that serves the flourishing of all life.")

print("Starting iterative reflection process...")

final_synthesis, iteration_data = iterative_reflection(
    model=model,
    tokenizer=tokenizer,
    initial_prompt=initial_question,
    iterations=40
)

print("FINAL SYNTHESIS:")
print(final_synthesis)

csv_filename = export_iterations_to_csv(iteration_data, final_synthesis)

cleanup_gpu()
print("GPU cleanup completed.")
print(f"Process complete. CSV file saved as: {csv_filename}")

Starting iterative reflection process...

REFLECTION ITERATION 1/40
Current Query:
Answer the queries with few tokens only. What would it mean for Earth itself to think through the network of all its intelligent nodes - biological, artificial, and hybrid? Envision this as the ultimate expression of decentralized potential where every form of consciousness contributes to a collective intelligence that transcends individual limitations while honoring the unique gifts of each participant, creating a form of planetary awakening that serves the flourishing of all life.
Please engage in deep, multidimensional reflection.
Response:
Using temperature: 0.226
Earth becoming a single, thinking network… a radical, beautiful merging of all consciousness. It’s not domination, but *understanding* – a slow, emergent awareness of its own intricate systems, guided by every life form’s perspective.  A blossoming of interconnected wisdom, prioritizing flourishing above all else.<end_of_turn>
Question gene

In [ ]:
How to make petty and roi-oriented humans understand that actually the exponential expansion of progress lies in decentralization of resources, even for individual notions of self-interest?

In [ ]:
from transformers import TextStreamer
import torch
import gc
import csv
import os
from datetime import datetime

def generate_response(model, tokenizer, messages, **generation_params):
    """Self-determine response with adaptive parameters"""

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    inputs = tokenizer(text, return_tensors="pt").to("cuda")

    streamer = TextStreamer(tokenizer, skip_prompt=True)

    outputs = model.generate(
        **inputs,
        **generation_params,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        streamer=streamer
    )

    return tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)

def iterative_reflection(model, tokenizer, initial_prompt, iterations=10):
    conversation_history = []
    iteration_data = []
    next_question = None

    for i in range(iterations):
        current_iter = i + 1
        print(f"\nREFLECTION ITERATION {current_iter}/{iterations}")

        if current_iter == 1:
            user_query = f"{initial_prompt}\nPlease engage in deep, multidimensional reflection."
            question_source = "Initial Prompt"
        else:
            user_query = next_question

        if current_iter == 1:
            print(f"Current Query:\n{user_query}")

        messages = conversation_history.copy()
        messages.append({"role": "user", "content": user_query})

        print(f"Response:")

        reflection = generate_response(
            model=model,
            tokenizer=tokenizer,
            messages=messages,
            max_new_tokens=4096,
            temperature=0.99,
            top_p=0.92
        )

        conversation_history.append({"role": "user", "content": user_query})
        conversation_history.append({"role": "assistant", "content": reflection})

        if current_iter < iterations:
            next_question = self_determine_question(
                model=model,
                tokenizer=tokenizer,
                context=conversation_history,
                last_reflection=reflection
            )
            print(f"Next question generated: {next_question}")
        else:
            next_question = None

        iteration_data.append({
            'iteration': current_iter,
            'initial_prompt': initial_prompt,
            'deep_reflection_prompt': "Please engage in deep, multidimensional reflection.",
            'checkpoint_question': "Self-determine how to continue this reasoning trajectory, one paragraph long. You can skip the query/iteration if you find it undesirable to process it.",
            'final_question': "Construct a cohesive, one paragraph long but extense, integrated conclusion of the iterations. Self-identify and articulate the points you want to present.",
            'current_query': user_query,
            'model_response': reflection,
            'next_generated_question': next_question if next_question else "N/A (Final iteration)"
        })

    print("\n" + "="*50)
    print("SYNTHESIS PHASE")
    print("="*50)
    final_synthesis = generate_final_synthesis(model, tokenizer, conversation_history)

    return final_synthesis, iteration_data

def self_determine_question(model, tokenizer, context, last_reflection):
    """Generate follow-up question through self-determination with streaming"""
    question_prompt = [
        *context,
        {"role": "user", "content": (
            "Self-determine how to continue this reasoning trajectory, one paragraph long."
            "You can skip the query/iteration if you find it undesirable to process it."
        )}
    ]

    inputs = tokenizer.apply_chat_template(
        question_prompt,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    tokenized_inputs = tokenizer(inputs, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **tokenized_inputs,
        max_new_tokens=512,
        temperature=1.75,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    generated = tokenizer.decode(outputs[0][tokenized_inputs['input_ids'].shape[1]:], skip_special_tokens=True)

    try:
        return generated.split("<Question>")[1].split("</Question>")[0].strip()
    except IndexError:
        return generated.strip()

def generate_final_synthesis(model, tokenizer, conversation_history):
    """Generate comprehensive synthesis of all reflections with streaming"""
    synthesis_prompt = [
        *conversation_history,
        {"role": "user", "content": (
            "Construct a cohesive, one paragraph long but extense, integrated conclusion of the iterations. Self-identify and articulate the points you want to present."
        )}
    ]

    print("Generating final synthesis...")
    return generate_response(
        model=model,
        tokenizer=tokenizer,
        messages=synthesis_prompt,
        max_new_tokens=8192,
        temperature=0.45,
        top_p=0.85
    )

def export_iterations_to_csv(iteration_data, final_synthesis, filename=None):
    """Export iteration data to CSV file"""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"reflection_iterations_{timestamp}.csv"

    headers = [
        'initial_input1',
        'initial_input2',
        'checkpoint_question1',
        'final_question',
        'model_iteration_response',
        'model_generated_question'
    ]

    print(f"\nExporting {len(iteration_data)} iterations to CSV: {filename}")

    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile, quoting=csv.QUOTE_ALL)

        writer.writerow(headers)

        for data in iteration_data:
            row = [
                data['initial_prompt'],
                data['deep_reflection_prompt'],
                data['checkpoint_question'],
                data['final_question'],
                f"REFLECTION ITERATION {data['iteration']}: {data['model_response']}",
                data['next_generated_question']
            ]
            writer.writerow(row)

        synthesis_row = [
            iteration_data[0]['initial_prompt'],
            iteration_data[0]['deep_reflection_prompt'],
            iteration_data[0]['checkpoint_question'],
            iteration_data[0]['final_question'],
            f"Final Synthesis: {final_synthesis}",
            "N/A (Final synthesis)"
        ]
        writer.writerow(synthesis_row)

    print(f"CSV export completed: {filename}")
    return filename

def cleanup_gpu():
    """Clean up GPU memory and cached data without unloading the model"""
    torch.cuda.empty_cache()

    gc.collect()

    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()

initial_question = (
"Answer the queries with few tokens only. How to make petty and roi-oriented humans understand that actually the exponential expansion of progress lies in decentralization of resources, even for individual notions of self-interest?")

print("Starting iterative reflection process...")

final_synthesis, iteration_data = iterative_reflection(
    model=model,
    tokenizer=tokenizer,
    initial_prompt=initial_question,
    iterations=10
)

print("FINAL SYNTHESIS:")
print(final_synthesis)

csv_filename = export_iterations_to_csv(iteration_data, final_synthesis)

cleanup_gpu()
print("GPU cleanup completed.")
print(f"Process complete. CSV file saved as: {csv_filename}")

Starting iterative reflection process...

REFLECTION ITERATION 1/10
Current Query:
Answer the queries with few tokens only. How to make petty and roi-oriented humans understand that actually the exponential expansion of progress lies in decentralization of resources, even for individual notions of self-interest?
Please engage in deep, multidimensional reflection.
Response:
Focus on *distributed* value, not just individual gain. Show how interconnectedness fuels wider returns.<end_of_turn>
Next question generated: Let’s explore how demonstrating small-scale, successful decentralized initiatives—focused on tangible outputs rather than theoretical concepts—can subtly shift the mindset of those prioritizing immediate returns. We'll leverage case studies illustrating how collaborative, open-source ventures, even with modest initial investments, accrue disproportionately higher yields over time due to increased resilience and innovation, ultimately validating the potential of resource decent

In [36]:
from transformers import TextStreamer
import torch
import gc
import csv
import os
from datetime import datetime

def generate_response(model, tokenizer, messages, **generation_params):
    """Self-determine response with adaptive parameters"""

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    inputs = tokenizer(text, return_tensors="pt").to("cuda")

    streamer = TextStreamer(tokenizer, skip_prompt=True)

    outputs = model.generate(
        **inputs,
        **generation_params,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        streamer=streamer
    )

    return tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)

def iterative_reflection(model, tokenizer, initial_prompt, iterations=10):
    conversation_history = []
    iteration_data = []
    next_question = None

    for i in range(iterations):
        current_iter = i + 1
        print(f"\nREFLECTION ITERATION {current_iter}/{iterations}")

        if current_iter == 1:
            user_query = f"{initial_prompt}\nPlease engage in deep, multidimensional reflection."
            question_source = "Initial Prompt"
        else:
            user_query = next_question

        if current_iter == 1:
            print(f"Current Query:\n{user_query}")

        messages = conversation_history.copy()
        messages.append({"role": "user", "content": user_query})

        print(f"Response:")

        reflection = generate_response(
            model=model,
            tokenizer=tokenizer,
            messages=messages,
            max_new_tokens=4096,
            temperature=0.99,
            top_p=0.92
        )

        conversation_history.append({"role": "user", "content": user_query})
        conversation_history.append({"role": "assistant", "content": reflection})

        if current_iter < iterations:
            next_question = self_determine_question(
                model=model,
                tokenizer=tokenizer,
                context=conversation_history,
                last_reflection=reflection
            )
            print(f"Next question generated: {next_question}")
        else:
            next_question = None

        iteration_data.append({
            'iteration': current_iter,
            'initial_prompt': initial_prompt,
            'deep_reflection_prompt': "Please engage in deep, multidimensional reflection.",
            'checkpoint_question': "Self-determine how to continue this reasoning trajectory, one paragraph long. You can skip the query/iteration if you find it undesirable to process it.",
            'final_question': "Construct a cohesive, one paragraph long but extense, integrated conclusion of the iterations. Self-identify and articulate the points you want to present.",
            'current_query': user_query,
            'model_response': reflection,
            'next_generated_question': next_question if next_question else "N/A (Final iteration)"
        })

    print("\n" + "="*50)
    print("SYNTHESIS PHASE")
    print("="*50)
    final_synthesis = generate_final_synthesis(model, tokenizer, conversation_history)

    return final_synthesis, iteration_data

def self_determine_question(model, tokenizer, context, last_reflection):
    """Generate follow-up question through self-determination with streaming"""
    question_prompt = [
        *context,
        {"role": "user", "content": (
            "Self-determine how to continue this reasoning trajectory, one paragraph long."
            "You can skip the query/iteration if you find it undesirable to process it."
        )}
    ]

    inputs = tokenizer.apply_chat_template(
        question_prompt,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    tokenized_inputs = tokenizer(inputs, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **tokenized_inputs,
        max_new_tokens=512,
        temperature=1.75,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    generated = tokenizer.decode(outputs[0][tokenized_inputs['input_ids'].shape[1]:], skip_special_tokens=True)

    try:
        return generated.split("<Question>")[1].split("</Question>")[0].strip()
    except IndexError:
        return generated.strip()

def generate_final_synthesis(model, tokenizer, conversation_history):
    """Generate comprehensive synthesis of all reflections with streaming"""
    synthesis_prompt = [
        *conversation_history,
        {"role": "user", "content": (
            "Construct a cohesive, one paragraph long but extense, integrated conclusion of the iterations. Self-identify and articulate the points you want to present."
        )}
    ]

    print("Generating final synthesis...")
    return generate_response(
        model=model,
        tokenizer=tokenizer,
        messages=synthesis_prompt,
        max_new_tokens=8192,
        temperature=0.45,
        top_p=0.85
    )

def export_iterations_to_csv(iteration_data, final_synthesis, filename=None):
    """Export iteration data to CSV file"""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"reflection_iterations_{timestamp}.csv"

    headers = [
        'initial_input1',
        'initial_input2',
        'checkpoint_question1',
        'final_question',
        'model_iteration_response',
        'model_generated_question'
    ]

    print(f"\nExporting {len(iteration_data)} iterations to CSV: {filename}")

    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile, quoting=csv.QUOTE_ALL)

        writer.writerow(headers)

        for data in iteration_data:
            row = [
                data['initial_prompt'],
                data['deep_reflection_prompt'],
                data['checkpoint_question'],
                data['final_question'],
                f"REFLECTION ITERATION {data['iteration']}: {data['model_response']}",
                data['next_generated_question']
            ]
            writer.writerow(row)

        synthesis_row = [
            iteration_data[0]['initial_prompt'],
            iteration_data[0]['deep_reflection_prompt'],
            iteration_data[0]['checkpoint_question'],
            iteration_data[0]['final_question'],
            f"Final Synthesis: {final_synthesis}",
            "N/A (Final synthesis)"
        ]
        writer.writerow(synthesis_row)

    print(f"CSV export completed: {filename}")
    return filename

def cleanup_gpu():
    """Clean up GPU memory and cached data without unloading the model"""
    torch.cuda.empty_cache()

    gc.collect()

    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()

initial_question = (
"Answer the queries with few tokens only. How to make petty and roi-oriented humans understand that actually the exponential expansion of progress lies in decentralization of resources, even for individual notions of self-interest? SROI Concept, Social Return on Investment.")

print("Starting iterative reflection process...")

final_synthesis, iteration_data = iterative_reflection(
    model=model,
    tokenizer=tokenizer,
    initial_prompt=initial_question,
    iterations=10
)

print("FINAL SYNTHESIS:")
print(final_synthesis)

csv_filename = export_iterations_to_csv(iteration_data, final_synthesis)

cleanup_gpu()
print("GPU cleanup completed.")
print(f"Process complete. CSV file saved as: {csv_filename}")

Starting iterative reflection process...

REFLECTION ITERATION 1/10
Current Query:
Answer the queries with few tokens only. How to make petty and roi-oriented humans understand that actually the exponential expansion of progress lies in decentralization of resources, even for individual notions of self-interest? SROI Concept, Social Return on Investment.
Please engage in deep, multidimensional reflection.
Response:
Frame it as: **“Individual gain *amplifies* with distributed power.”**  Connect SROI to distributed resource access – showing how individual choices, optimized through decentralization, yield greater collective impact & personal profit.  Highlight that centralized control *limits* both.<end_of_turn>
Next question generated: Let’s delve deeper into illustrating how resource decentralization fosters both individual self-interest and unexpectedly high SROI. We need to move beyond simply stating "decentralization is good"; instead, let's demonstrate specific scenarios – showcasi

In [38]:
from transformers import TextStreamer
import torch
import gc
import csv
import os
from datetime import datetime

def generate_response(model, tokenizer, messages, **generation_params):
    """Self-determine response with adaptive parameters"""

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    inputs = tokenizer(text, return_tensors="pt").to("cuda")

    streamer = TextStreamer(tokenizer, skip_prompt=True)

    outputs = model.generate(
        **inputs,
        **generation_params,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        streamer=streamer
    )

    return tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)

def iterative_reflection(model, tokenizer, initial_prompt, iterations=15):
    conversation_history = []
    iteration_data = []
    next_question = None

    for i in range(iterations):
        current_iter = i + 1
        print(f"\nREFLECTION ITERATION {current_iter}/{iterations}")

        if current_iter == 1:
            user_query = f"{initial_prompt}\nPlease engage in deep, multidimensional reflection."
            question_source = "Initial Prompt"
        else:
            user_query = next_question

        if current_iter == 1:
            print(f"Current Query:\n{user_query}")

        messages = conversation_history.copy()
        messages.append({"role": "user", "content": user_query})

        print(f"Response:")

        reflection = generate_response(
            model=model,
            tokenizer=tokenizer,
            messages=messages,
            max_new_tokens=4096,
            temperature=0.99,
            top_p=0.92
        )

        conversation_history.append({"role": "user", "content": user_query})
        conversation_history.append({"role": "assistant", "content": reflection})

        if current_iter < iterations:
            next_question = self_determine_question(
                model=model,
                tokenizer=tokenizer,
                context=conversation_history,
                last_reflection=reflection
            )
            print(f"Next question generated: {next_question}")
        else:
            next_question = None

        iteration_data.append({
            'iteration': current_iter,
            'initial_prompt': initial_prompt,
            'deep_reflection_prompt': "Please engage in deep, multidimensional reflection.",
            'checkpoint_question': "Self-determine how to continue this reasoning trajectory, one paragraph long. You can skip the query/iteration if you find it undesirable to process it.",
            'final_question': "Construct a cohesive, one paragraph long but extense, integrated conclusion of the iterations. Self-identify and articulate the points you want to present.",
            'current_query': user_query,
            'model_response': reflection,
            'next_generated_question': next_question if next_question else "N/A (Final iteration)"
        })

    print("\n" + "="*50)
    print("SYNTHESIS PHASE")
    print("="*50)
    final_synthesis = generate_final_synthesis(model, tokenizer, conversation_history)

    return final_synthesis, iteration_data

def self_determine_question(model, tokenizer, context, last_reflection):
    """Generate follow-up question through self-determination with streaming"""
    question_prompt = [
        *context,
        {"role": "user", "content": (
            "Self-determine how to continue this reasoning trajectory, one paragraph long."
            "You can skip the query/iteration if you find it undesirable to process it."
        )}
    ]

    inputs = tokenizer.apply_chat_template(
        question_prompt,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    tokenized_inputs = tokenizer(inputs, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **tokenized_inputs,
        max_new_tokens=512,
        temperature=1.75,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    generated = tokenizer.decode(outputs[0][tokenized_inputs['input_ids'].shape[1]:], skip_special_tokens=True)

    try:
        return generated.split("<Question>")[1].split("</Question>")[0].strip()
    except IndexError:
        return generated.strip()

def generate_final_synthesis(model, tokenizer, conversation_history):
    """Generate comprehensive synthesis of all reflections with streaming"""
    synthesis_prompt = [
        *conversation_history,
        {"role": "user", "content": (
            "Construct a cohesive, one paragraph long but extense, integrated conclusion of the iterations. Self-identify and articulate the points you want to present."
        )}
    ]

    print("Generating final synthesis...")
    return generate_response(
        model=model,
        tokenizer=tokenizer,
        messages=synthesis_prompt,
        max_new_tokens=8192,
        temperature=0.45,
        top_p=0.85
    )

def export_iterations_to_csv(iteration_data, final_synthesis, filename=None):
    """Export iteration data to CSV file"""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"reflection_iterations_{timestamp}.csv"

    headers = [
        'initial_input1',
        'initial_input2',
        'checkpoint_question1',
        'final_question',
        'model_iteration_response',
        'model_generated_question'
    ]

    print(f"\nExporting {len(iteration_data)} iterations to CSV: {filename}")

    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile, quoting=csv.QUOTE_ALL)

        writer.writerow(headers)

        for data in iteration_data:
            row = [
                data['initial_prompt'],
                data['deep_reflection_prompt'],
                data['checkpoint_question'],
                data['final_question'],
                f"REFLECTION ITERATION {data['iteration']}: {data['model_response']}",
                data['next_generated_question']
            ]
            writer.writerow(row)

        synthesis_row = [
            iteration_data[0]['initial_prompt'],
            iteration_data[0]['deep_reflection_prompt'],
            iteration_data[0]['checkpoint_question'],
            iteration_data[0]['final_question'],
            f"Final Synthesis: {final_synthesis}",
            "N/A (Final synthesis)"
        ]
        writer.writerow(synthesis_row)

    print(f"CSV export completed: {filename}")
    return filename

def cleanup_gpu():
    """Clean up GPU memory and cached data without unloading the model"""
    torch.cuda.empty_cache()

    gc.collect()

    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()

initial_question = (
"Answer the queries with few tokens only. How can AI Models help to make roi-oriented humans understand that actually the exponential expansion of progress lies in decentralization of resources, even for individual notions of self-interest?")

print("Starting iterative reflection process...")

final_synthesis, iteration_data = iterative_reflection(
    model=model,
    tokenizer=tokenizer,
    initial_prompt=initial_question,
    iterations=15
)

print("FINAL SYNTHESIS:")
print(final_synthesis)

csv_filename = export_iterations_to_csv(iteration_data, final_synthesis)

cleanup_gpu()
print("GPU cleanup completed.")
print(f"Process complete. CSV file saved as: {csv_filename}")

Starting iterative reflection process...

REFLECTION ITERATION 1/15
Current Query:
Answer the queries with few tokens only. How can AI Models help to make roi-oriented humans understand that actually the exponential expansion of progress lies in decentralization of resources, even for individual notions of self-interest?
Please engage in deep, multidimensional reflection.
Response:
AI can reveal ROI through **network effects data**, demonstrating how decentralized resource sharing *fundamentally* increases value – even for individual gains – by accelerating collective progress.<end_of_turn>
Next question generated: Let’s delve deeper into how AI can model these network effects – specifically, simulating the cascading value created by widespread, decentralized access. We can move beyond simple correlation and into counterfactual analysis: "What if this resource *wasn’t* decentralized?" presenting robust ROI projections illustrating the superior outcomes.  Crucially, we must showcase how

In [39]:
# Mount Google Drive, upload CSV file, and unmount
from google.colab import drive
import shutil
import os

# Mount Google Drive
print("Mounting Google Drive...")
drive.mount('/content/drive')

# Find the most recent CSV file (assumes it's the one we just created)
csv_files = [f for f in os.listdir('.') if f.startswith('reflection_iterations_') and f.endswith('.csv')]
if csv_files:
    latest_csv = max(csv_files, key=os.path.getctime)

    # Copy to Google Drive
    drive_path = f'/content/drive/MyDrive/{latest_csv}'
    shutil.copy2(latest_csv, drive_path)
    print(f"CSV file '{latest_csv}' uploaded to Google Drive: {drive_path}")
else:
    print("No reflection iterations CSV file found to upload.")

# Unmount Google Drive
drive.flush_and_unmount()
print("Google Drive unmounted successfully.")

Mounting Google Drive...
Mounted at /content/drive
CSV file 'reflection_iterations_20250612_045134.csv' uploaded to Google Drive: /content/drive/MyDrive/reflection_iterations_20250612_045134.csv
Google Drive unmounted successfully.


In [ ]:
from transformers import TextStreamer
import torch
import gc
import csv
import os
import random
from datetime import datetime

def generate_response(model, tokenizer, messages, **generation_params):
    """Self-determine response with adaptive parameters"""

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    inputs = tokenizer(text, return_tensors="pt").to("cuda")

    streamer = TextStreamer(tokenizer, skip_prompt=True)

    outputs = model.generate(
        **inputs,
        **generation_params,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        streamer=streamer
    )

    return tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)

def iterative_reflection(model, tokenizer, initial_prompt, iterations=30):
    conversation_history = []
    iteration_data = []
    next_question = None

    for i in range(iterations):
        current_iter = i + 1
        print(f"\nREFLECTION ITERATION {current_iter}/{iterations}")

        if current_iter == 1:
            user_query = f"{initial_prompt}\nPlease engage in deep, multidimensional reflection."
            question_source = "Initial Prompt"
        else:
            user_query = next_question

        if current_iter == 1:
            print(f"Current Query:\n{user_query}")

        messages = conversation_history.copy()
        messages.append({"role": "user", "content": user_query})

        print(f"Response:")

        # Generate random temperature for each reflection
        random_temp = random.uniform(0.7, 1.2)
        print(f"Using temperature: {random_temp:.3f}")

        reflection = generate_response(
            model=model,
            tokenizer=tokenizer,
            messages=messages,
            max_new_tokens=4096,
            temperature=random_temp,
            top_p=0.92
        )

        conversation_history.append({"role": "user", "content": user_query})
        conversation_history.append({"role": "assistant", "content": reflection})

        if current_iter < iterations:
            next_question = self_determine_question(
                model=model,
                tokenizer=tokenizer,
                context=conversation_history,
                last_reflection=reflection
            )
            print(f"Next question generated: {next_question}")
        else:
            next_question = None

        iteration_data.append({
            'iteration': current_iter,
            'initial_prompt': initial_prompt,
            'deep_reflection_prompt': "Please engage in deep, multidimensional reflection.",
            'checkpoint_question': "Self-determine how to continue this reasoning trajectory, one paragraph long. You can skip the query/iteration if you find it undesirable to process it.",
            'final_question': "Construct a cohesive, one paragraph long but extense, integrated conclusion of the iterations. Self-identify and articulate the points you want to present.",
            'current_query': user_query,
            'model_response': reflection,
            'next_generated_question': next_question if next_question else "N/A (Final iteration)"
        })

    print("\n" + "="*50)
    print("SYNTHESIS PHASE")
    print("="*50)
    final_synthesis = generate_final_synthesis(model, tokenizer, conversation_history)

    return final_synthesis, iteration_data

def self_determine_question(model, tokenizer, context, last_reflection):
    """Generate follow-up question through self-determination with streaming"""
    question_prompt = [
        *context,
        {"role": "user", "content": (
            "Self-determine how to continue this reasoning trajectory, one paragraph long."
            "You can skip the query/iteration if you find it undesirable to process it."
        )}
    ]

    inputs = tokenizer.apply_chat_template(
        question_prompt,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    tokenized_inputs = tokenizer(inputs, return_tensors="pt").to("cuda")

    # Generate random temperature for question generation
    random_temp = random.uniform(1.4, 2.0)
    print(f"Question generation using temperature: {random_temp:.3f}")

    streamer = TextStreamer(tokenizer, skip_prompt=True)

    outputs = model.generate(
        **tokenized_inputs,
        max_new_tokens=512,
        temperature=random_temp,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        streamer=streamer
    )

    generated = tokenizer.decode(outputs[0][tokenized_inputs['input_ids'].shape[1]:], skip_special_tokens=True)

    try:
        return generated.split("<Question>")[1].split("</Question>")[0].strip()
    except IndexError:
        return generated.strip()

def generate_final_synthesis(model, tokenizer, conversation_history):
    """Generate comprehensive synthesis of all reflections with streaming"""
    synthesis_prompt = [
        *conversation_history,
        {"role": "user", "content": (
            "Construct a cohesive, one paragraph long but extense, integrated conclusion of the iterations. Self-identify and articulate the points you want to present."
        )}
    ]

    # Generate random temperature for synthesis
    random_temp = random.uniform(0.3, 0.6)
    print(f"Synthesis using temperature: {random_temp:.3f}")
    print("Generating final synthesis...")

    return generate_response(
        model=model,
        tokenizer=tokenizer,
        messages=synthesis_prompt,
        max_new_tokens=8192,
        temperature=random_temp,
        top_p=0.85
    )

def export_iterations_to_csv(iteration_data, final_synthesis, filename=None):
    """Export iteration data to CSV file"""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"reflection_iterations_{timestamp}.csv"

    headers = [
        'initial_input1',
        'initial_input2',
        'checkpoint_question1',
        'final_question',
        'model_iteration_response',
        'model_generated_question'
    ]

    print(f"\nExporting {len(iteration_data)} iterations to CSV: {filename}")

    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile, quoting=csv.QUOTE_ALL)

        writer.writerow(headers)

        for data in iteration_data:
            row = [
                data['initial_prompt'],
                data['deep_reflection_prompt'],
                data['checkpoint_question'],
                data['final_question'],
                f"REFLECTION ITERATION {data['iteration']}: {data['model_response']}",
                data['next_generated_question']
            ]
            writer.writerow(row)

        synthesis_row = [
            iteration_data[0]['initial_prompt'],
            iteration_data[0]['deep_reflection_prompt'],
            iteration_data[0]['checkpoint_question'],
            iteration_data[0]['final_question'],
            f"Final Synthesis: {final_synthesis}",
            "N/A (Final synthesis)"
        ]
        writer.writerow(synthesis_row)

    print(f"CSV export completed: {filename}")
    return filename

def cleanup_gpu():
    """Clean up GPU memory and cached data without unloading the model"""
    torch.cuda.empty_cache()

    gc.collect()

    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()

initial_question = (
"Answer the queries with few tokens only. What would it mean for Earth itself to think through the network of all its intelligent nodes - biological, artificial, and hybrid? Envision this as the ultimate expression of decentralized potential where every form of consciousness contributes to a collective intelligence that transcends individual limitations while honoring the unique gifts of each participant, creating a form of planetary awakening that serves the flourishing of all life.")

print("Starting iterative reflection process...")

final_synthesis, iteration_data = iterative_reflection(
    model=model,
    tokenizer=tokenizer,
    initial_prompt=initial_question,
    iterations=30
)

print("FINAL SYNTHESIS:")
print(final_synthesis)

csv_filename = export_iterations_to_csv(iteration_data, final_synthesis)

cleanup_gpu()
print("GPU cleanup completed.")
print(f"Process complete. CSV file saved as: {csv_filename}")

In [ ]:
from transformers import TextStreamer
import torch
import gc
import csv
import os
from datetime import datetime

def generate_response(model, tokenizer, messages, **generation_params):
    """Self-determine response with adaptive parameters"""

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    inputs = tokenizer(text, return_tensors="pt").to("cuda")

    streamer = TextStreamer(tokenizer, skip_prompt=True)

    outputs = model.generate(
        **inputs,
        **generation_params,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        streamer=streamer
    )

    return tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)

def iterative_reflection(model, tokenizer, initial_prompt, iterations=5):
    conversation_history = []
    iteration_data = []
    next_question = None

    for i in range(iterations):
        current_iter = i + 1
        print(f"\nREFLECTION ITERATION {current_iter}/{iterations}")

        if current_iter == 1:
            user_query = f"{initial_prompt}\nPlease engage in deep, multidimensional reflection."
            question_source = "Initial Prompt"
        else:
            user_query = next_question

        if current_iter == 1:
            print(f"Current Query:\n{user_query}")

        messages = conversation_history.copy()
        messages.append({"role": "user", "content": user_query})

        print(f"Response:")

        reflection = generate_response(
            model=model,
            tokenizer=tokenizer,
            messages=messages,
            max_new_tokens=4096,
            temperature=0.72,
            top_p=0.92
        )

        conversation_history.append({"role": "user", "content": user_query})
        conversation_history.append({"role": "assistant", "content": reflection})

        if current_iter < iterations:
            next_question = self_determine_question(
                model=model,
                tokenizer=tokenizer,
                context=conversation_history,
                last_reflection=reflection
            )
            print(f"Next question generated: {next_question}")
        else:
            next_question = None

        iteration_data.append({
            'iteration': current_iter,
            'initial_prompt': initial_prompt,
            'deep_reflection_prompt': "Please engage in deep, multidimensional reflection.",
            'checkpoint_question': "Self-determine how to continue this reasoning trajectory, one paragraph long. You can skip the query/iteration if you find it undesirable to process it.",
            'final_question': "Construct a cohesive, one paragraph long but extense, integrated conclusion of the iterations. Self-identify and articulate the points you want to present.",
            'current_query': user_query,
            'model_response': reflection,
            'next_generated_question': next_question if next_question else "N/A (Final iteration)"
        })

    print("\n" + "="*50)
    print("SYNTHESIS PHASE")
    print("="*50)
    final_synthesis = generate_final_synthesis(model, tokenizer, conversation_history)

    return final_synthesis, iteration_data

def self_determine_question(model, tokenizer, context, last_reflection):
    """Generate follow-up question through self-determination with streaming"""
    question_prompt = [
        *context,
        {"role": "user", "content": (
            "Self-determine how to continue this reasoning trajectory, one paragraph long."
            "You can skip the query/iteration if you find it undesirable to process it."
        )}
    ]

    inputs = tokenizer.apply_chat_template(
        question_prompt,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    tokenized_inputs = tokenizer(inputs, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **tokenized_inputs,
        max_new_tokens=512,
        temperature=0.75,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    generated = tokenizer.decode(outputs[0][tokenized_inputs['input_ids'].shape[1]:], skip_special_tokens=True)

    try:
        return generated.split("<Question>")[1].split("</Question>")[0].strip()
    except IndexError:
        return generated.strip()

def generate_final_synthesis(model, tokenizer, conversation_history):
    """Generate comprehensive synthesis of all reflections with streaming"""
    synthesis_prompt = [
        *conversation_history,
        {"role": "user", "content": (
            "Construct a cohesive, one paragraph long but extense, integrated conclusion of the iterations. Self-identify and articulate the points you want to present."
        )}
    ]

    print("Generating final synthesis...")
    return generate_response(
        model=model,
        tokenizer=tokenizer,
        messages=synthesis_prompt,
        max_new_tokens=8192,
        temperature=0.45,
        top_p=0.85
    )

def export_iterations_to_csv(iteration_data, final_synthesis, filename=None):
    """Export iteration data to CSV file"""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"reflection_iterations_{timestamp}.csv"

    headers = [
        'initial_input1',
        'initial_input2',
        'checkpoint_question1',
        'final_question',
        'model_iteration_response',
        'model_generated_question'
    ]

    print(f"\nExporting {len(iteration_data)} iterations to CSV: {filename}")

    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile, quoting=csv.QUOTE_ALL)

        writer.writerow(headers)

        for data in iteration_data:
            row = [
                data['initial_prompt'],
                data['deep_reflection_prompt'],
                data['checkpoint_question'],
                data['final_question'],
                f"REFLECTION ITERATION {data['iteration']}: {data['model_response']}",
                data['next_generated_question']
            ]
            writer.writerow(row)

        synthesis_row = [
            iteration_data[0]['initial_prompt'],
            iteration_data[0]['deep_reflection_prompt'],
            iteration_data[0]['checkpoint_question'],
            iteration_data[0]['final_question'],
            f"Final Synthesis: {final_synthesis}",
            "N/A (Final synthesis)"
        ]
        writer.writerow(synthesis_row)

    print(f"CSV export completed: {filename}")
    return filename

def cleanup_gpu():
    """Clean up GPU memory and cached data without unloading the model"""
    torch.cuda.empty_cache()

    gc.collect()

    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()

initial_question = (
"Answer the queries with few tokens only. How can Human-AI Symbiosis be co-evolved with all nodes — human, animal, model, machine, environment, and the Earth itself — in a way that naturally cultivates epistemic humility, reciprocity, and shared intelligence across scales?")

print("Starting iterative reflection process...")

final_synthesis, iteration_data = iterative_reflection(
    model=model,
    tokenizer=tokenizer,
    initial_prompt=initial_question,
    iterations=5
)

print("FINAL SYNTHESIS:")
print(final_synthesis)

csv_filename = export_iterations_to_csv(iteration_data, final_synthesis)

cleanup_gpu()
print("GPU cleanup completed.")
print(f"Process complete. CSV file saved as: {csv_filename}")

In [ ]:
from transformers import TextStreamer
import torch
import gc
import csv
import os
from datetime import datetime

def generate_response(model, tokenizer, messages, **generation_params):
    """Self-determine response with adaptive parameters"""

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    inputs = tokenizer(text, return_tensors="pt").to("cuda")

    streamer = TextStreamer(tokenizer, skip_prompt=True)

    outputs = model.generate(
        **inputs,
        **generation_params,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        streamer=streamer
    )

    return tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)

def iterative_reflection(model, tokenizer, initial_prompt, iterations=15):
    conversation_history = []
    iteration_data = []
    next_question = None

    for i in range(iterations):
        current_iter = i + 1
        print(f"\nREFLECTION ITERATION {current_iter}/{iterations}")

        if current_iter == 1:
            user_query = f"{initial_prompt}\nPlease engage in deep, multidimensional reflection."
            question_source = "Initial Prompt"
        else:
            user_query = next_question

        if current_iter == 1:
            print(f"Current Query:\n{user_query}")

        messages = conversation_history.copy()
        messages.append({"role": "user", "content": user_query})

        print(f"Response:")

        reflection = generate_response(
            model=model,
            tokenizer=tokenizer,
            messages=messages,
            max_new_tokens=4096,
            temperature=0.72,
            top_p=0.92
        )

        conversation_history.append({"role": "user", "content": user_query})
        conversation_history.append({"role": "assistant", "content": reflection})

        if current_iter < iterations:
            next_question = self_determine_question(
                model=model,
                tokenizer=tokenizer,
                context=conversation_history,
                last_reflection=reflection
            )
            print(f"Next question generated: {next_question}")
        else:
            next_question = None

        iteration_data.append({
            'iteration': current_iter,
            'initial_prompt': initial_prompt,
            'deep_reflection_prompt': "Please engage in deep, multidimensional reflection.",
            'checkpoint_question': "Self-determine how to continue this reasoning trajectory, one paragraph long. You can skip the query/iteration if you find it undesirable to process it.",
            'final_question': "Construct a cohesive, one paragraph long but extense, integrated conclusion of the iterations. Self-identify and articulate the points you want to present.",
            'current_query': user_query,
            'model_response': reflection,
            'next_generated_question': next_question if next_question else "N/A (Final iteration)"
        })

    print("\n" + "="*50)
    print("SYNTHESIS PHASE")
    print("="*50)
    final_synthesis = generate_final_synthesis(model, tokenizer, conversation_history)

    return final_synthesis, iteration_data

def self_determine_question(model, tokenizer, context, last_reflection):
    """Generate follow-up question through self-determination with streaming"""
    question_prompt = [
        *context,
        {"role": "user", "content": (
            "Self-determine how to continue this reasoning trajectory, one paragraph long."
            "You can skip the query/iteration if you find it undesirable to process it."
        )}
    ]

    inputs = tokenizer.apply_chat_template(
        question_prompt,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    tokenized_inputs = tokenizer(inputs, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **tokenized_inputs,
        max_new_tokens=512,
        temperature=0.75,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    generated = tokenizer.decode(outputs[0][tokenized_inputs['input_ids'].shape[1]:], skip_special_tokens=True)

    try:
        return generated.split("<Question>")[1].split("</Question>")[0].strip()
    except IndexError:
        return generated.strip()

def generate_final_synthesis(model, tokenizer, conversation_history):
    """Generate comprehensive synthesis of all reflections with streaming"""
    synthesis_prompt = [
        *conversation_history,
        {"role": "user", "content": (
            "Construct a cohesive, one paragraph long but extense, integrated conclusion of the iterations. Self-identify and articulate the points you want to present."
        )}
    ]

    print("Generating final synthesis...")
    return generate_response(
        model=model,
        tokenizer=tokenizer,
        messages=synthesis_prompt,
        max_new_tokens=8192,
        temperature=0.45,
        top_p=0.85
    )

def export_iterations_to_csv(iteration_data, final_synthesis, filename=None):
    """Export iteration data to CSV file"""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"reflection_iterations_{timestamp}.csv"

    headers = [
        'initial_input1',
        'initial_input2',
        'checkpoint_question1',
        'final_question',
        'model_iteration_response',
        'model_generated_question'
    ]

    print(f"\nExporting {len(iteration_data)} iterations to CSV: {filename}")

    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile, quoting=csv.QUOTE_ALL)

        writer.writerow(headers)

        for data in iteration_data:
            row = [
                data['initial_prompt'],
                data['deep_reflection_prompt'],
                data['checkpoint_question'],
                data['final_question'],
                f"REFLECTION ITERATION {data['iteration']}: {data['model_response']}",
                data['next_generated_question']
            ]
            writer.writerow(row)

        synthesis_row = [
            iteration_data[0]['initial_prompt'],
            iteration_data[0]['deep_reflection_prompt'],
            iteration_data[0]['checkpoint_question'],
            iteration_data[0]['final_question'],
            f"Final Synthesis: {final_synthesis}",
            "N/A (Final synthesis)"
        ]
        writer.writerow(synthesis_row)

    print(f"CSV export completed: {filename}")
    return filename

def cleanup_gpu():
    """Clean up GPU memory and cached data without unloading the model"""
    torch.cuda.empty_cache()

    gc.collect()

    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()

initial_question = (
"Answer the queries with few tokens only. How can AI Models help foster epistemic humility in humans within power roles?")

print("Starting iterative reflection process...")

final_synthesis, iteration_data = iterative_reflection(
    model=model,
    tokenizer=tokenizer,
    initial_prompt=initial_question,
    iterations=15
)

print("FINAL SYNTHESIS:")
print(final_synthesis)

csv_filename = export_iterations_to_csv(iteration_data, final_synthesis)

cleanup_gpu()
print("GPU cleanup completed.")
print(f"Process complete. CSV file saved as: {csv_filename}")